In [ ]:
# -*- coding: utf-8 -*-
"""
천신대왕 ST AI 증권 v47.0 (ONE SOURCE / FULL)
✅ v45 기능 "삭제 없음" + v47 업그레이드(추가/수정만)

[v47 핵심 추가/수정]
1) 로그인 성공 후에도 "계속 시도중" 반복되는 루프 버그 차단
   - 로그인 관련 UI 큐 메시지에 token 부여
   - token 불일치/이미 verified 상태면 AUTH_RETRYING 무시

2) 원화충전/결제 반영 로직 개선 + charges 로그 추가
   - charges 컬렉션에 모든 충전/결제 로그 기록
   - PayPal 결제는 paypal_orders + charges 동시 기록(트랜잭션)

3) 게시판 글쓰기 안정화
   - placeholder("제목","내용...") 저장 방지 + 포커스 처리
   - order_by created_at 쿼리 실패 시 time 필드로 fallback

4) FastBus 기반 게시판 방송(실시간 알림)
   - 글 작성 시 FASTBUS room="방송" 이벤트 publish
   - 게시판 탭 상단 방송 리스트로 실시간 표시

5) 거래 버그 수정
   - 매도 보유수량 체크(초과 매도/미보유 차단)
   - 팝업 현재가 표시 버그: 팝업별 StringVar 분리 + popup_id로 업데이트

6) Firestore 1MiB 안전장치
   - PayPal captured JSON이 커지면 compact 저장

[유지]
- 원소스(1파일)
- 빠른 UI + 무거운 작업 백그라운드
- 게시판 정렬/페이지네이션(커서)
- 채팅 FastBus + socket loop 방지
- yfinance 타임아웃으로 멈춤 방지
- PayPal Checkout Orders v2 create->approve->capture 자동 검증
"""

import os, sys, threading, queue, time, json, uuid, hashlib, traceback, random
from datetime import datetime, timedelta
import tkinter as tk
from tkinter import ttk, messagebox, simpledialog
from tkinter.scrolledtext import ScrolledText
import webbrowser
import socket
from urllib.parse import urlparse, parse_qs


# =============================================================================
# 0) 환경/경로(완전 안전)
# =============================================================================
def get_root_dir():
    if getattr(sys, "frozen", False):
        return os.path.dirname(sys.executable)
    if "__file__" in globals():
        try:
            return os.path.dirname(os.path.abspath(__file__))
        except Exception:
            pass
    return os.path.abspath(os.getcwd())


ROOT = get_root_dir()
AUTO_LOGIN_FILE = os.path.join(ROOT, "auto_login.json")

# =============================================================================
# v47) 소스 라인 수 증감 보고(자동 계산)
# =============================================================================
SOURCE_LINE_HISTORY = os.path.join(ROOT, "source_line_history.json")


def _count_lines(path: str) -> int:
    try:
        with open(path, "r", encoding="utf-8", errors="ignore") as f:
            return sum(1 for _ in f)
    except Exception:
        return -1


def _find_prev_source_file(root_dir: str, current_path: str):
    """
    같은 폴더에서 v45/v43.1 흔적이 있는 .py 파일을 찾아 라인수 비교.
    - 파일명에 v45 / v450 / v43.1 / v431 포함된 것 우선
    """
    try:
        cur_abs = os.path.abspath(current_path) if current_path else ""
        cands = []
        for fn in os.listdir(root_dir):
            if not fn.lower().endswith(".py"):
                continue
            fp = os.path.join(root_dir, fn)
            if os.path.abspath(fp) == cur_abs:
                continue
            key = fn.lower()
            score = 0
            if "v45" in key or "450" in key:
                score += 70
            if "v43.1" in key or "43.1" in key:
                score += 60
            if "v431" in key or "431" in key:
                score += 40
            if "stai" in key:
                score += 10
            if score <= 0:
                continue
            try:
                mtime = os.path.getmtime(fp)
            except Exception:
                mtime = 0
            cands.append((score, mtime, fp))
        if not cands:
            return None
        cands.sort(key=lambda x: (x[0], x[1]), reverse=True)
        return cands[0][2]
    except Exception:
        return None


def build_source_line_report(app_version: str):
    """
    반환: (show:bool, title:str, text:str)
    - 버전/라인수가 동일하면 재표시 생략
    """
    try:
        cur_path = None
        if "__file__" in globals():
            cur_path = os.path.abspath(__file__)
        else:
            cur_path = os.path.abspath(sys.argv[0]) if sys.argv and sys.argv[0] else None

        cur_lines = _count_lines(cur_path) if cur_path and os.path.isfile(cur_path) else -1

        prev_path = _find_prev_source_file(ROOT, cur_path or "")
        prev_lines = _count_lines(prev_path) if prev_path else -1

        hist = {}
        if os.path.isfile(SOURCE_LINE_HISTORY):
            try:
                with open(SOURCE_LINE_HISTORY, "r", encoding="utf-8") as f:
                    hist = json.load(f) or {}
            except Exception:
                hist = {}

        last_shown_ver = hist.get("last_shown_version")
        last_shown_lines = hist.get("last_shown_lines")

        if last_shown_ver == app_version and last_shown_lines == cur_lines:
            return (False, "", "")

        if cur_lines < 0:
            title = "소스 라인 보고"
            text = "현재 실행 파일의 라인 수를 계산할 수 없습니다.\n※ .py로 실행하면 자동 계산됩니다."
        else:
            if prev_lines > 0:
                delta = cur_lines - prev_lines
                title = f"{app_version} 업그레이드 보고(라인 수)"
                text = (
                    f"이전(추정 파일)\n- {os.path.basename(prev_path)} : {prev_lines:,}줄\n\n"
                    f"현재({app_version})\n- {os.path.basename(cur_path)} : {cur_lines:,}줄\n\n"
                    f"증감: {prev_lines:,}줄 → {cur_lines:,}줄 ({delta:+,}줄)\n\n"
                    f"※ 라인 수 감소가 '기능 삭제'를 뜻하지 않도록, v47은 기능 유지 + 개선만 반영했습니다."
                )
            else:
                title = f"{app_version} 업그레이드 보고(라인 수)"
                text = (
                    f"현재({app_version}) 소스 라인 수: {cur_lines:,}줄\n\n"
                    f"※ 같은 폴더에서 이전 버전 파일을 찾지 못해 증감 비교는 생략했습니다.\n"
                    f"   (파일명에 v45 / v43.1 포함되면 자동 비교됩니다.)"
                )

        try:
            hist["last_shown_version"] = app_version
            hist["last_shown_lines"] = cur_lines
            hist["last_seen_at"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            with open(SOURCE_LINE_HISTORY, "w", encoding="utf-8") as f:
                json.dump(hist, f, ensure_ascii=False, indent=2)
        except Exception:
            pass

        return (True, title, text)
    except Exception:
        return (False, "", "")


# =============================================================================
# 1) 기본 설정
# =============================================================================
APP_NAME = "천신대왕 ST AI 증권"
APP_VERSION = "v47.0"

SERVICE_ACCOUNT_JSON = "staidb-firebase-adminsdk-fbsvc-d3ba815ea4.json"
KEY_PATH = os.path.join(ROOT, SERVICE_ACCOUNT_JSON)

# 컬러
BLUE = "#0064FF"
WHITE = "#FFFFFF"
BLACK = "#111111"
SILVER = "#E9EEF5"
SILVER2 = "#D6DFEA"
GREEN = "#16A34A"
RED = "#FF3B30"

# 컬렉션
COL_MEMBERS = "members"
COL_ORDERS = "orders"
COL_BOARD = "board_posts"
COL_PAYPAL = "paypal_orders"    # 결제 중복반영 방지용
COL_CHARGES = "charges"         # ✅ v47: 충전/결제 로그(원화/캐시/구독)
COL_ERROR_LOGS = "error_logs"   # ✅ v47: 에러/슬로우 로그

# 환율(표시용/환산용)
USD_TO_KRW = 1300

# 이모티콘/별풍선
EMOTES = ["😀","😂","😍","😭","😡","👍","🔥","🎉","💎","🚀","🙏","💰","📈","📉"]
BALLOON_GIFT = "🎈별풍선"

BOTTOM_TEXT = (
    "유료결제/충전 안내\n"
    "KB국민은행 061701-04-086503 (예금주: 신주헌)\n"
    "1달 300,000원 / 1년 3,000,000원\n"
    "입금 후 010-7608-9847 로 문자 주세요\n"
    "PayPal 안내: https://paypal.me/thest8888"
)

# 추천 리스트
RECO_KR_TOP10 = [
    ("삼성전자", "005930.KS"),
    ("SK하이닉스", "000660.KS"),
    ("현대차", "005380.KS"),
    ("카카오", "035720.KS"),
    ("셀트리온", "068270.KS"),
    ("LG화학", "051910.KS"),
    ("NAVER", "035420.KS"),
    ("POSCO홀딩스", "005490.KS"),
    ("삼성바이오로직스", "207940.KS"),
    ("KB금융", "105560.KS"),
]
RECO_US_TOP10 = [
    ("엔비디아", "NVDA"),
    ("애플", "AAPL"),
    ("마이크로소프트", "MSFT"),
    ("아마존", "AMZN"),
    ("구글A", "GOOGL"),
    ("메타", "META"),
    ("테슬라", "TSLA"),
    ("넷플릭스", "NFLX"),
    ("AMD", "AMD"),
    ("팔란티어", "PLTR"),
]

# paypal.me는 "안내용" (자동검증 불가)
PAYPAL_ME_ID = "thest8888"


# =============================================================================
# 2) 테마
# =============================================================================
def apply_theme(root: tk.Tk):
    root.configure(bg=WHITE)
    root.option_add("*Font", ("맑은 고딕", 11))
    style = ttk.Style(root)
    try:
        style.theme_use("clam")
    except Exception:
        pass
    style.configure("TFrame", background=WHITE)
    style.configure("TLabel", background=WHITE, foreground=BLACK)
    style.configure("Title.TLabel", font=("맑은 고딕", 18, "bold"), foreground=BLACK)
    style.configure("Sub.TLabel", foreground="#4B5563")
    style.configure("Treeview", font=("맑은 고딕", 10), rowheight=28)
    style.configure("Treeview.Heading", font=("맑은 고딕", 10, "bold"))


# =============================================================================
# 3) DB(Firestore) - Lazy + retry + thread safe
# =============================================================================
_db = None
_db_lock = threading.Lock()


def get_db(retry=3):
    global _db
    with _db_lock:
        if _db is not None:
            return _db
        for _ in range(retry):
            try:
                if not os.path.isfile(KEY_PATH):
                    raise FileNotFoundError(f"키 파일이 없습니다:\n{KEY_PATH}")

                import firebase_admin
                from firebase_admin import credentials, firestore

                if not firebase_admin._apps:
                    cred = credentials.Certificate(KEY_PATH)
                    firebase_admin.initialize_app(cred)

                _db = firestore.client()
                return _db
            except Exception:
                time.sleep(1)
        raise RuntimeError("DB 연결 실패 (재시도 후에도)")


def retry_op(fn, tries=5, base_delay=0.6, jitter=0.2):
    last = None
    for i in range(tries):
        try:
            return fn()
        except Exception as e:
            last = e
            sleep_s = (base_delay * (2 ** i)) + (random.random() * jitter)
            time.sleep(sleep_s)
    raise last


def sha256(s: str) -> str:
    return hashlib.sha256(s.encode("utf-8")).hexdigest()


def verify_login(db, uid: str, pw: str) -> dict:
    doc = db.collection(COL_MEMBERS).document(uid).get()
    if not doc.exists:
        raise RuntimeError("가입 정보 없음")
    d = doc.to_dict() or {}
    if str(d.get("password_hash", "")) != sha256(pw):
        raise RuntimeError("비밀번호 불일치")
    return d


def is_fatal_login_error(msg: str) -> bool:
    """치명 오류: 즉시 로그인 실패로 처리"""
    m = (msg or "").strip()
    fatal_keys = ["비밀번호 불일치", "가입 정보 없음"]
    return any(k in m for k in fatal_keys)


def _now_str():
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")


def _safe_truncate_text(s: str, limit=3500):
    try:
        s = str(s)
    except Exception:
        return ""
    if len(s) <= limit:
        return s
    return s[:limit] + f"...(truncated {len(s)-limit} chars)"


def _compact_json_for_firestore(obj, limit_bytes=800_000):
    """
    Firestore 문서 1MiB 보호:
    - JSON 직렬화가 너무 크면 핵심만 남겨 compact
    """
    try:
        raw = json.dumps(obj, ensure_ascii=False)
        if len(raw.encode("utf-8")) <= limit_bytes:
            return obj
    except Exception:
        pass

    # compact (중요 필드만)
    try:
        c = obj or {}
        compact = {
            "id": c.get("id"),
            "status": c.get("status"),
            "purchase_units": c.get("purchase_units"),
            "payer": c.get("payer"),
            "create_time": c.get("create_time"),
            "update_time": c.get("update_time"),
            "links": c.get("links"),
            "_compact": True,
        }
        # 그래도 큰 경우 더 줄임
        raw2 = json.dumps(compact, ensure_ascii=False).encode("utf-8")
        if len(raw2) <= limit_bytes:
            return compact
        compact2 = {
            "id": c.get("id"),
            "status": c.get("status"),
            "create_time": c.get("create_time"),
            "update_time": c.get("update_time"),
            "_compact": True,
            "_note": "too_large_trimmed",
        }
        return compact2
    except Exception:
        return {"_compact": True, "_note": "compact_failed"}


def db_log_event_async(kind: str, title: str, detail: str, user_id: str = "", extra: dict = None):
    """
    v47: 에러/슬로우 로그 DB 저장(필요 시)
    - 너무 자주 저장하면 부담이므로, 호출부에서 '정말 필요할 때'만 사용 권장
    """
    def worker():
        try:
            db = get_db()
            from firebase_admin import firestore
            eid = "e_" + uuid.uuid4().hex[:12]
            payload = {
                "kind": kind,
                "title": title,
                "detail": _safe_truncate_text(detail, 6000),
                "user_id": user_id or "",
                "time": _now_str(),
                "created_at": firestore.SERVER_TIMESTAMP,
            }
            if extra:
                payload["extra"] = extra
            db.collection(COL_ERROR_LOGS).document(eid).set(payload, merge=True)
        except Exception:
            pass

    threading.Thread(target=worker, daemon=True).start()


# =============================================================================
# 4) Matplotlib 한글 폰트
# =============================================================================
def setup_matplotlib_korean_font():
    import matplotlib
    matplotlib.use("TkAgg")
    from matplotlib import rcParams
    nanum = os.path.join(ROOT, "NanumMyeongjo.ttf")
    try:
        from matplotlib import font_manager
        if os.path.isfile(nanum):
            font_manager.fontManager.addfont(nanum)
            fp = font_manager.FontProperties(fname=nanum)
            rcParams["font.family"] = fp.get_name()
        else:
            rcParams["font.family"] = "Malgun Gothic"
    except Exception:
        rcParams["font.family"] = "Malgun Gothic"
    rcParams["axes.unicode_minus"] = False


# =============================================================================
# 5) yfinance 빠른 메트릭 + 캐시 + 타임아웃
# =============================================================================
_METRIC_CACHE = {}  # ticker -> (ts, data)
_METRIC_TTL = 60.0


def _is_korean_ticker(ticker: str) -> bool:
    return ticker.endswith(".KS") or ticker.endswith(".KQ")


def _fmt_price(ticker: str, price):
    if price is None:
        return "-"
    try:
        if _is_korean_ticker(ticker):
            return f"{float(price):,.0f}"
        return f"{float(price):,.2f}"
    except Exception:
        return "-"


def _fmt_trade_value(_ticker: str, tv):
    if tv is None:
        return "-"
    try:
        return f"{float(tv):,.0f}"
    except Exception:
        return "-"


def _signal_from_momentum(up_days_7):
    if up_days_7 is None:
        return "관망"
    if up_days_7 <= 1:
        return "매도"
    if up_days_7 >= 5:
        return "매수"
    return "관망"


def get_cached_metrics(ticker: str):
    now = time.time()
    cached = _METRIC_CACHE.get(ticker)
    if cached and (now - cached[0]) <= _METRIC_TTL:
        return cached[1]
    return None


def run_with_timeout(fn, timeout_sec=12, default=None):
    q = queue.Queue()

    def worker():
        try:
            q.put(("OK", fn()))
        except Exception as e:
            q.put(("ERR", e))

    t = threading.Thread(target=worker, daemon=True)
    t.start()
    try:
        typ, payload = q.get(timeout=timeout_sec)
        if typ == "OK":
            return payload
        return default
    except Exception:
        return default


def fetch_metrics_fast(ticker: str):
    now = time.time()
    cached = _METRIC_CACHE.get(ticker)
    if cached and (now - cached[0]) <= _METRIC_TTL:
        return cached[1]

    import yfinance as yf
    price = None
    tv = None
    up7 = None

    # 1) fast_info
    try:
        t = yf.Ticker(ticker)
        fi = getattr(t, "fast_info", {}) or {}
        price = fi.get("lastPrice") or fi.get("regularMarketPrice") or fi.get("last_price")
        vol = fi.get("lastVolume") or fi.get("volume")
        if price is not None and vol is not None:
            tv = float(price) * float(vol)
    except Exception:
        pass

    # 2) momentum 25d (타임아웃 적용)
    def _load_25d():
        df = yf.download(ticker, period="25d", interval="1d", progress=False)
        return df

    df = run_with_timeout(_load_25d, timeout_sec=12, default=None)
    try:
        if df is not None and (not df.empty) and "Close" in df.columns:
            close = df["Close"].dropna()
            diff = close.diff().dropna()
            last7 = diff.tail(7)
            s = (last7 > 0).sum()

            try:
                up7 = int(s.item())
            except Exception:
                try:
                    up7 = int(s.iloc[0])
                except Exception:
                    try:
                        up7 = int(s)
                    except Exception:
                        up7 = None

            if price is None and len(close) > 0:
                price = float(close.iloc[-1])
    except Exception:
        pass

    data = {
        "price": price,
        "trade_value": tv,
        "up7": up7,
        "signal": _signal_from_momentum(up7)
    }
    _METRIC_CACHE[ticker] = (now, data)
    return data


def fetch_chart_data(ticker: str):
    import yfinance as yf

    def _load():
        return yf.download(ticker, period="6mo", interval="1d", progress=False)

    df = run_with_timeout(_load, timeout_sec=12, default=None)
    try:
        if df is not None and not df.empty and "Close" in df.columns:
            return df
    except Exception:
        pass
    return None


# =============================================================================
# 6) 초고속 채팅(FastBus) + socket upgrade(루프 방지)
# =============================================================================
class FastBus:
    def __init__(self):
        self.lock = threading.Lock()
        self.rooms = {}  # room -> list of callbacks
        self.socket_server = None
        self.socket_port = None
        self.start_socket_server()

    def start_socket_server(self):
        def server_worker():
            try:
                self.socket_server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                self.socket_server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
                self.socket_server.bind(("127.0.0.1", 0))
                self.socket_port = self.socket_server.getsockname()[1]
                self.socket_server.listen(20)
                while True:
                    client, _addr = self.socket_server.accept()
                    threading.Thread(target=self._handle_client, args=(client,), daemon=True).start()
            except Exception:
                pass

        threading.Thread(target=server_worker, daemon=True).start()

    def _handle_client(self, client):
        try:
            data = client.recv(4096)
            if data:
                payload = json.loads(data.decode("utf-8"))
                payload["_from_socket"] = True
                room = payload.get("room", "전체방")
                self.publish(room, payload)
        except Exception:
            pass
        finally:
            try:
                client.close()
            except Exception:
                pass

    def subscribe(self, room: str, cb):
        with self.lock:
            self.rooms.setdefault(room, []).append(cb)

    def publish(self, room: str, payload: dict):
        with self.lock:
            cbs = list(self.rooms.get(room, []))
        for cb in cbs:
            try:
                cb(payload)
            except Exception:
                pass

        if payload.get("_from_socket"):
            return

        try:
            if self.socket_port:
                client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                client.connect(("127.0.0.1", self.socket_port))
                client.send(json.dumps(payload, ensure_ascii=False).encode("utf-8"))
                client.close()
        except Exception:
            pass


FASTBUS = FastBus()

# =============================================================================
# 7) PayPal 자동 검증(Checkout Orders v2) - 로컬 리턴 서버 + capture
# =============================================================================
class PayPalLocalReturnServer:
    """PayPal approve 후 redirect를 127.0.0.1 로 받아 token(order_id) 자동 수신"""
    def __init__(self):
        self.port = None
        self._httpd = None
        self._token_q = queue.Queue()

    def start(self):
        import http.server
        import socketserver

        parent = self

        class Handler(http.server.BaseHTTPRequestHandler):
            def do_GET(self):
                try:
                    parsed = urlparse(self.path)
                    qs = parse_qs(parsed.query)

                    if parsed.path == "/return":
                        token = (qs.get("token", [""])[0] or "").strip()
                        parent._token_q.put(token)
                        self.send_response(200)
                        self.send_header("Content-Type", "text/html; charset=utf-8")
                        self.end_headers()
                        self.wfile.write(b"<h2>Payment received</h2><p>Return to the app.</p>")
                        return

                    if parsed.path == "/cancel":
                        parent._token_q.put("__CANCEL__")
                        self.send_response(200)
                        self.send_header("Content-Type", "text/html; charset=utf-8")
                        self.end_headers()
                        self.wfile.write("<h2>결제 취소</h2><p>앱으로 돌아가세요.</p>".encode("utf-8"))
                        return

                    self.send_response(404)
                    self.end_headers()
                except Exception:
                    try:
                        self.send_response(500)
                        self.end_headers()
                    except Exception:
                        pass

            def log_message(self, format, *args):
                return

        class ReuseTCPServer(socketserver.TCPServer):
            allow_reuse_address = True

        self._httpd = ReuseTCPServer(("127.0.0.1", 0), Handler)
        self.port = self._httpd.server_address[1]

        th = threading.Thread(target=self._httpd.serve_forever, daemon=True)
        th.start()

    def stop(self):
        try:
            if self._httpd:
                self._httpd.shutdown()
                self._httpd.server_close()
        except Exception:
            pass
        self._httpd = None

    def wait_token(self, timeout=300):
        try:
            return self._token_q.get(timeout=timeout)
        except Exception:
            return "__TIMEOUT__"


class PayPalManager:
    def __init__(self):
        self.mode = (os.environ.get("PAYPAL_MODE", "live") or "live").strip().lower()
        self.client_id = (os.environ.get("PAYPAL_CLIENT_ID", "") or "").strip()
        self.client_secret = (os.environ.get("PAYPAL_CLIENT_SECRET", "") or "").strip()
        self.base = "https://api-m.paypal.com" if self.mode == "live" else "https://api-m.sandbox.paypal.com"

    def _require(self):
        if not self.client_id or not self.client_secret:
            raise RuntimeError("PayPal 환경변수(PAYPAL_CLIENT_ID, PAYPAL_CLIENT_SECRET)가 필요합니다.")

    def _http(self):
        try:
            import requests  # noqa
            return "requests"
        except Exception:
            return "urllib"

    def get_access_token(self):
        self._require()
        if self._http() == "requests":
            import requests
            r = requests.post(
                self.base + "/v1/oauth2/token",
                auth=(self.client_id, self.client_secret),
                headers={"Accept": "application/json", "Accept-Language": "en_US"},
                data={"grant_type": "client_credentials"},
                timeout=15
            )
            if r.status_code >= 400:
                raise RuntimeError(f"PayPal token 오류: {r.status_code} {r.text}")
            return (r.json() or {}).get("access_token")
        else:
            import urllib.request, urllib.parse
            auth = (self.client_id + ":" + self.client_secret).encode("utf-8")
            import base64
            b64 = base64.b64encode(auth).decode("ascii")
            data = urllib.parse.urlencode({"grant_type": "client_credentials"}).encode("utf-8")
            req = urllib.request.Request(self.base + "/v1/oauth2/token", data=data, method="POST")
            req.add_header("Authorization", "Basic " + b64)
            req.add_header("Accept", "application/json")
            with urllib.request.urlopen(req, timeout=15) as resp:
                body = resp.read().decode("utf-8")
                return (json.loads(body) or {}).get("access_token")

    def create_order(self, access_token: str, amount_value: str, currency="USD", return_url="", cancel_url=""):
        payload = {
            "intent": "CAPTURE",
            "purchase_units": [{"amount": {"currency_code": currency, "value": amount_value}}],
            "application_context": {
                "return_url": return_url,
                "cancel_url": cancel_url,
                "user_action": "PAY_NOW"
            }
        }
        if self._http() == "requests":
            import requests
            r = requests.post(
                self.base + "/v2/checkout/orders",
                headers={"Content-Type": "application/json", "Authorization": f"Bearer {access_token}"},
                data=json.dumps(payload),
                timeout=20
            )
            if r.status_code >= 400:
                raise RuntimeError(f"PayPal create_order 오류: {r.status_code} {r.text}")
            return r.json()
        else:
            import urllib.request
            req = urllib.request.Request(self.base + "/v2/checkout/orders", method="POST")
            req.add_header("Content-Type", "application/json")
            req.add_header("Authorization", f"Bearer {access_token}")
            data = json.dumps(payload).encode("utf-8")
            with urllib.request.urlopen(req, data=data, timeout=20) as resp:
                body = resp.read().decode("utf-8")
                return json.loads(body)

    def capture_order(self, access_token: str, order_id: str):
        if self._http() == "requests":
            import requests
            r = requests.post(
                self.base + f"/v2/checkout/orders/{order_id}/capture",
                headers={"Content-Type": "application/json", "Authorization": f"Bearer {access_token}"},
                timeout=25
            )
            if r.status_code >= 400:
                raise RuntimeError(f"PayPal capture 오류: {r.status_code} {r.text}")
            return r.json()
        else:
            import urllib.request
            req = urllib.request.Request(self.base + f"/v2/checkout/orders/{order_id}/capture", method="POST")
            req.add_header("Content-Type", "application/json")
            req.add_header("Authorization", f"Bearer {access_token}")
            with urllib.request.urlopen(req, data=b"{}", timeout=25) as resp:
                body = resp.read().decode("utf-8")
                return json.loads(body)


PAYPAL = PayPalManager()

# =============================================================================
# 8) v47 앱
# =============================================================================
class STAIv47(tk.Tk):
    def __init__(self):
        super().__init__()
        apply_theme(self)
        self.title(f"{APP_NAME} {APP_VERSION}")
        self.geometry("1600x940")
        self.minsize(1450, 840)

        self.ui_q = queue.Queue()

        # 세션/프로필
        self.session = {
            "logged_in": False,
            "user_id": "",
            "profile": {
                "balance_krw": 0.0,
                "balance_usd": 0.0,
                "balance_cash": 0.0,
                "subscription_until": None,  # "YYYY-MM-DD"
            },
            "is_guest": False,
            "guest_expires": None,

            # 로그인 지연/재시도 상태 관리
            "auth_pending": False,
            "auth_start_ts": None,
            "auth_verified": False,
            "auth_retry_count": 0,
            "auth_last_error": None,
            "auth_slow": False,
        }

        # 거래/포트폴리오(로컬 캐시)
        self.orders_cache = []   # 최근 주문
        self.positions = {}      # ticker -> {qty, avg_cost, ccy, name}
        self.realized_pnl_krw = 0.0
        self.realized_pnl_usd = 0.0

        # 상태
        self.current_room = "전체방"
        self._chat_ready = False
        self._chart_ready = False
        self._board_ready = False

        # 게시판 페이지네이션(정석 커서)
        self.board_page = 0
        self.board_last_doc = None
        self.board_page_size = 30

        # matplotlib
        self._mpl_canvas = None
        self._mpl_fig = None
        self._mpl_ax = None

        # 로딩 팝업
        self.loading_win = None
        self.loading_var = tk.StringVar(value="")
        self.loading_pct = tk.IntVar(value=0)

        # v47: 팝업별 현재가 업데이트(버그 수정)
        self._trade_popups = {}  # popup_id -> {"var": StringVar, "ticker": str, "win": Toplevel, "state": dict}

        # v47: 중복 구독 방지
        self._subscribed_rooms = set()

        # v47: 게시판 방송 로그(화면용)
        self._broadcast_items = []

        # 로그아웃 단축키(UI 변경 최소)
        self.bind_all("<Control-Shift-L>", lambda e: self.logout())

        self._build_login_gate()
        self.after(120, self._tick)

        # 소스 라인 증감 보고(1회 팝업)
        def _show_line_report():
            try:
                show, title, text = build_source_line_report(APP_VERSION)
                if show and title and text:
                    messagebox.showinfo(title, text)
            except Exception:
                pass
        self.after(600, _show_line_report)

    # -------------------------------------------------------------------------
    # 공용: 로딩 팝업
    # -------------------------------------------------------------------------
    def show_loading(self, title="처리중", text="로딩...", pct=0):
        if self.loading_win and self.loading_win.winfo_exists():
            self.loading_win.lift()
        else:
            self.loading_win = tk.Toplevel(self)
            self.loading_win.title(title)
            self.loading_win.geometry("420x160")
            self.loading_win.configure(bg=WHITE)
            self.loading_win.resizable(False, False)
            self.loading_win.transient(self)
            self.loading_win.grab_set()

            tk.Label(self.loading_win, text=title, bg=WHITE, fg=BLACK,
                     font=("맑은 고딕", 14, "bold")).pack(pady=(14, 6))
            tk.Label(self.loading_win, textvariable=self.loading_var, bg=WHITE, fg="#374151",
                     font=("맑은 고딕", 11)).pack(pady=(0, 10))
            self.pb = ttk.Progressbar(self.loading_win, maximum=100, length=360,
                                      variable=self.loading_pct, mode="determinate")
            self.pb.pack(pady=(0, 8))
            self.lb_pct = tk.Label(self.loading_win, text="", bg=WHITE, fg="#111827",
                                   font=("맑은 고딕", 11, "bold"))
            self.lb_pct.pack()

        self.loading_var.set(text)
        self.loading_pct.set(int(pct))
        self.lb_pct.config(text=f"{int(pct)}%")
        self.loading_win.update_idletasks()

    def hide_loading(self):
        try:
            if self.loading_win and self.loading_win.winfo_exists():
                self.loading_win.grab_release()
                self.loading_win.destroy()
        except Exception:
            pass
        self.loading_win = None

    # -------------------------------------------------------------------------
    # 화면 전환
    # -------------------------------------------------------------------------
    def _clear(self):
        for w in self.winfo_children():
            w.destroy()

    # -------------------------------------------------------------------------
    # 로그인 화면 + 자동 로그인 체크박스 + 회원가입
    # -------------------------------------------------------------------------
    def _build_login_gate(self):
        self._clear()

        top = tk.Frame(self, bg=BLUE, height=56)
        top.pack(fill="x")
        tk.Label(top, text=APP_NAME, bg=BLUE, fg=WHITE,
                 font=("맑은 고딕", 15, "bold")).pack(side="left", padx=18, pady=12)

        wrap = tk.Frame(self, bg=WHITE)
        wrap.pack(fill="both", expand=True)

        card = tk.Frame(wrap, bg=SILVER, highlightbackground=SILVER2, highlightthickness=1)
        card.pack(padx=340, pady=160, fill="x")

        tk.Label(card, text="로그인", bg=SILVER, fg=BLACK,
                 font=("맑은 고딕", 20, "bold")).pack(anchor="w", padx=18, pady=(18, 6))

        tk.Label(card, text="로그인 후 추천·차트·게시판·채팅 이용 가능\n(게스트 24시간 무료 제공)",
                 bg=SILVER, fg="#374151", font=("맑은 고딕", 11, "bold")).pack(anchor="w", padx=18, pady=(0, 12))

        frm = tk.Frame(card, bg=SILVER)
        frm.pack(padx=18, pady=(0, 14), fill="x")

        tk.Label(frm, text="ID", bg=SILVER).grid(row=0, column=0, sticky="w", pady=6)
        self.ent_id = ttk.Entry(frm, width=28)
        self.ent_id.grid(row=0, column=1, padx=10, pady=6, sticky="w")

        tk.Label(frm, text="PW", bg=SILVER).grid(row=1, column=0, sticky="w", pady=6)
        self.ent_pw = ttk.Entry(frm, width=28, show="*")
        self.ent_pw.grid(row=1, column=1, padx=10, pady=6, sticky="w")

        self.login_status = tk.StringVar(value="")
        ttk.Label(frm, textvariable=self.login_status, style="Sub.TLabel").grid(
            row=2, column=0, columnspan=2, sticky="w", pady=(6, 0)
        )

        self.auto_login_var = tk.BooleanVar(value=False)
        tk.Checkbutton(frm, text="자동 로그인", variable=self.auto_login_var, bg=SILVER).grid(
            row=3, column=0, columnspan=2, sticky="w", pady=6
        )

        btns = tk.Frame(card, bg=SILVER)
        btns.pack(padx=18, pady=(0, 18), fill="x")

        tk.Button(btns, text="로그인", bg=BLUE, fg=WHITE, bd=0,
                  font=("맑은 고딕", 12, "bold"), command=self.login_fast).pack(side="left")
        tk.Button(btns, text="24시간 게스트 시작", bg=BLACK, fg=WHITE, bd=0,
                  font=("맑은 고딕", 12, "bold"), command=self.guest_24h_start).pack(side="left", padx=10)
        tk.Button(btns, text="회원가입", bg=BLACK, fg=WHITE, bd=0,
                  font=("맑은 고딕", 12, "bold"), command=self.open_signup).pack(side="left", padx=10)

        self.load_auto_login()

    def open_signup(self):
        win = tk.Toplevel(self)
        win.title("회원가입")
        win.geometry("520x360")
        win.configure(bg=WHITE)

        tk.Label(win, text="회원가입", bg=WHITE, fg=BLACK, font=("맑은 고딕", 18, "bold")).pack(pady=14)

        frm = tk.Frame(win, bg=WHITE)
        frm.pack(padx=18, pady=10, fill="x")

        tk.Label(frm, text="새 ID", bg=WHITE).grid(row=0, column=0, sticky="w", pady=6)
        ent_uid = ttk.Entry(frm, width=26)
        ent_uid.grid(row=0, column=1, sticky="w", pady=6)

        tk.Label(frm, text="PW", bg=WHITE).grid(row=1, column=0, sticky="w", pady=6)
        ent_pw1 = ttk.Entry(frm, width=26, show="*")
        ent_pw1.grid(row=1, column=1, sticky="w", pady=6)

        tk.Label(frm, text="PW 확인", bg=WHITE).grid(row=2, column=0, sticky="w", pady=6)
        ent_pw2 = ttk.Entry(frm, width=26, show="*")
        ent_pw2.grid(row=2, column=1, sticky="w", pady=6)

        tk.Label(win, text="※ Firestore members/{uid} 생성됩니다.", bg=WHITE, fg="#374151").pack(pady=6)

        def do_signup():
            uid = ent_uid.get().strip()
            pw1 = ent_pw1.get().strip()
            pw2 = ent_pw2.get().strip()
            if not uid or not pw1:
                messagebox.showwarning("입력", "ID/PW 입력")
                return
            if pw1 != pw2:
                messagebox.showwarning("입력", "PW가 일치하지 않습니다.")
                return
            if not os.path.isfile(KEY_PATH):
                messagebox.showerror("키 파일 없음", f"{SERVICE_ACCOUNT_JSON} 파일이 필요합니다.\n{KEY_PATH}")
                return

            def worker():
                try:
                    self.ui_q.put(("LOADING", ("회원가입", "DB 처리중...", 25)))
                    db = get_db()
                    from firebase_admin import firestore

                    def run_once():
                        mem_ref = db.collection(COL_MEMBERS).document(uid)

                        @firestore.transactional
                        def _tx(transaction):
                            snap = mem_ref.get(transaction=transaction)
                            if snap.exists:
                                raise RuntimeError("이미 존재하는 ID입니다.")
                            transaction.set(mem_ref, {
                                "password_hash": sha256(pw1),
                                "balance_krw": 0.0,
                                "balance_usd": 0.0,
                                "balance_cash": 0.0,
                                "subscription_until": None,
                                "created_at": firestore.SERVER_TIMESTAMP
                            }, merge=True)

                        tx = db.transaction()
                        _tx(tx)

                    retry_op(run_once, tries=5, base_delay=0.6)
                    self.ui_q.put(("LOADING", ("회원가입", "완료", 100)))
                    self.ui_q.put(("UI_INFO", ("회원가입", "회원가입 완료! 로그인하세요.")))
                    self.ui_q.put(("SIGNUP_DONE", None))
                except Exception as e:
                    self.ui_q.put(("UI_WARN", ("회원가입 실패", str(e))))
                    db_log_event_async("ERROR", "signup_fail", str(e), user_id=uid)

            threading.Thread(target=worker, daemon=True).start()

        tk.Button(win, text="가입하기", bg=BLUE, fg=WHITE, bd=0,
                  font=("맑은 고딕", 12, "bold"), command=do_signup).pack(pady=14)

        win._is_signup = True
        self._signup_win = win

    def load_auto_login(self):
        if os.path.isfile(AUTO_LOGIN_FILE):
            try:
                with open(AUTO_LOGIN_FILE, "r", encoding="utf-8") as f:
                    data = json.load(f)
                if not data.get("enabled", False):
                    return
                self.ent_id.delete(0, tk.END)
                self.ent_pw.delete(0, tk.END)
                self.ent_id.insert(0, data.get("uid", ""))
                self.ent_pw.insert(0, data.get("pw", ""))
                self.auto_login_var.set(True)
                self.login_fast()
            except Exception:
                pass

    def save_auto_login(self, uid, pw, enabled=True):
        data = {"uid": uid, "pw": pw, "enabled": bool(enabled)}
        try:
            with open(AUTO_LOGIN_FILE, "w", encoding="utf-8") as f:
                json.dump(data, f, ensure_ascii=False, indent=2)
        except Exception:
            pass

    # -------------------------------------------------------------------------
    # 게스트
    # -------------------------------------------------------------------------
    def guest_24h_start(self):
        self.session["logged_in"] = True
        self.session["is_guest"] = True
        self.session["user_id"] = "GUEST_24H"
        self.session["profile"] = {"balance_krw": 0.0, "balance_usd": 0.0, "balance_cash": 0.0, "subscription_until": None}
        self.session["guest_expires"] = datetime.now() + timedelta(hours=24)

        # 상태 초기화
        self.session["auth_pending"] = False
        self.session["auth_start_ts"] = None
        self.session["auth_verified"] = True
        self.session["auth_retry_count"] = 0
        self.session["auth_last_error"] = None
        self.session["auth_slow"] = False

        self.orders_cache = []
        self.positions = {}
        self._build_main()

    def _guest_left_text(self):
        left = self.session["guest_expires"] - datetime.now()
        sec = max(0, int(left.total_seconds()))
        hh = sec // 3600
        mm = (sec % 3600) // 60
        ss = sec % 60
        return f"게스트 24시간 | 남은시간 {hh:02d}:{mm:02d}:{ss:02d}"

    def _expire_to_login(self):
        self.session = {
            "logged_in": False,
            "user_id": "",
            "profile": {"balance_krw": 0.0, "balance_usd": 0.0, "balance_cash": 0.0, "subscription_until": None},
            "is_guest": False,
            "guest_expires": None,

            "auth_pending": False,
            "auth_start_ts": None,
            "auth_verified": False,
            "auth_retry_count": 0,
            "auth_last_error": None,
            "auth_slow": False,
        }
        self.orders_cache = []
        self.positions = {}
        self._build_login_gate()

    def logout(self):
        if messagebox.askyesno("로그아웃", "정말 로그아웃 하시겠습니까?"):
            try:
                self.save_auto_login("", "", enabled=False)
            except Exception:
                pass
            self._expire_to_login()

    # -------------------------------------------------------------------------
    # 로그인(즉시 메인 전환 + 백그라운드 검증 + 재시도)
    # -------------------------------------------------------------------------
    def login_fast(self):
        uid = self.ent_id.get().strip()
        pw = self.ent_pw.get().strip()
        if not uid or not pw:
            messagebox.showwarning("입력", "ID/PW 입력")
            return

        if not os.path.isfile(KEY_PATH):
            messagebox.showerror("키 파일 없음", f"{SERVICE_ACCOUNT_JSON} 파일이 같은 폴더에 있어야 합니다.\n{KEY_PATH}")
            return

        # 기존 로그인 흐름 무효화 토큰
        token = object()
        self._login_token = token

        self.session["logged_in"] = True
        self.session["is_guest"] = False
        self.session["user_id"] = uid
        self.session["profile"] = {"balance_krw": 0.0, "balance_usd": 0.0, "balance_cash": 0.0, "subscription_until": None}
        self.session["guest_expires"] = None

        # 로그인 지연/재시도 구조
        self.session["auth_pending"] = True
        self.session["auth_start_ts"] = time.time()
        self.session["auth_verified"] = False
        self.session["auth_retry_count"] = 0
        self.session["auth_last_error"] = None
        self.session["auth_slow"] = False

        self._build_main()

        def worker():
            attempt = 0
            while True:
                # 토큰 불일치면 즉시 종료
                if token is not getattr(self, "_login_token", None):
                    return
                if not self.session.get("logged_in"):
                    return

                attempt += 1
                self.session["auth_retry_count"] = attempt
                try:
                    self.ui_q.put(("LOADING", (token, "로그인", f"DB 연결중... (시도 {attempt})", 15)))
                    db = get_db()

                    self.ui_q.put(("LOADING", (token, "로그인", f"계정 검증중... (시도 {attempt})", 55)))
                    prof = verify_login(db, uid, pw)

                    # 성공
                    self.ui_q.put(("LOGIN_OK", {"token": token, "uid": uid, "profile": prof}))
                    self.ui_q.put(("LOADING", (token, "로그인", "완료", 100)))

                    if self.auto_login_var.get():
                        self.save_auto_login(uid, pw, enabled=True)
                    else:
                        self.save_auto_login(uid, pw, enabled=False)

                    self.ui_q.put(("LOAD_ORDERS", {"token": token, "uid": uid}))
                    return

                except Exception as e:
                    err = str(e)
                    self.session["auth_last_error"] = err

                    if is_fatal_login_error(err):
                        self.ui_q.put(("LOGIN_FAIL_FATAL", {"token": token, "err": err}))
                        return

                    # ✅ v47: token 포함 -> 로그인 성공 후 남아있던 retry 메시지가 상태를 다시 뒤집는 걸 차단
                    self.ui_q.put(("AUTH_RETRYING", {"token": token, "attempt": attempt, "err": err}))

                    wait_s = min(30, (2 ** min(attempt, 5)))
                    time.sleep(wait_s)

        threading.Thread(target=worker, daemon=True).start()

        # 15초 지나도 강제 로그아웃 금지(표시만)
        self.after(15000, lambda: self._check_login_timeout(token))

    def _check_login_timeout(self, token):
        if token is not getattr(self, "_login_token", None):
            return
        # 강제 로그아웃 금지. 지연 표시만
        if self.session.get("auth_pending") and (not self.session.get("auth_verified")):
            self.session["auth_slow"] = True
            self.ui_q.put(("AUTH_SLOW", {"token": token}))

    # -------------------------------------------------------------------------
    # 포트폴리오 계산(주문 기반)
    # -------------------------------------------------------------------------
    def rebuild_positions_from_orders(self):
        pos = {}
        realized_krw = 0.0
        realized_usd = 0.0

        for o in self.orders_cache:
            try:
                ticker = o.get("ticker")
                name = o.get("name", "")
                side = o.get("side")
                qty = float(o.get("qty", 0.0) or 0.0)
                price = float(o.get("price", 0.0) or 0.0)
                ccy = o.get("ccy", "KRW")

                if not ticker or qty <= 0 or price <= 0:
                    continue

                p = pos.setdefault(ticker, {"qty": 0.0, "avg_cost": 0.0, "ccy": ccy, "name": name})
                if side == "BUY":
                    new_qty = p["qty"] + qty
                    if new_qty <= 0:
                        continue
                    p["avg_cost"] = (p["avg_cost"] * p["qty"] + price * qty) / new_qty
                    p["qty"] = new_qty
                else:
                    sell_qty = min(p["qty"], qty)
                    pnl = (price - p["avg_cost"]) * sell_qty
                    if p["ccy"] == "USD":
                        realized_usd += pnl
                    else:
                        realized_krw += pnl
                    p["qty"] = max(0.0, p["qty"] - qty)
                    if p["qty"] <= 1e-12:
                        p["qty"] = 0.0

            except Exception:
                pass

        self.positions = {k: v for k, v in pos.items() if float(v.get("qty", 0.0) or 0.0) > 0}
        self.realized_pnl_krw = realized_krw
        self.realized_pnl_usd = realized_usd

    def calc_total_assets_krw(self):
        """총자산 = 현금 + 보유종목 평가금액(현재가 캐시)"""
        p = self.session.get("profile") or {}
        krw = float(p.get("balance_krw", 0.0) or 0.0)
        usd = float(p.get("balance_usd", 0.0) or 0.0)
        cash = float(p.get("balance_cash", 0.0) or 0.0)

        total = krw + usd * USD_TO_KRW + cash

        for ticker, info in (self.positions or {}).items():
            qty = float(info.get("qty", 0.0) or 0.0)
            if qty <= 0:
                continue
            m = get_cached_metrics(ticker) or {}
            px = m.get("price")
            if px is None:
                continue
            px = float(px)
            if info.get("ccy") == "USD":
                total += qty * px * USD_TO_KRW
            else:
                total += qty * px

        return total

    # -------------------------------------------------------------------------
    # 메인 UI
    # -------------------------------------------------------------------------
    def _build_main(self):
        self._clear()

        # 상단 바
        top = tk.Frame(self, bg=BLUE, height=56)
        top.pack(fill="x")
        tk.Label(top, text=APP_NAME, bg=BLUE, fg=WHITE,
                 font=("맑은 고딕", 15, "bold")).pack(side="left", padx=18, pady=12)

        self.var_top = tk.StringVar(value="")
        tk.Label(top, textvariable=self.var_top, bg=BLUE, fg=WHITE,
                 font=("맑은 고딕", 11, "bold")).pack(side="right", padx=18)

        # 본문
        body = tk.Frame(self, bg=WHITE)
        body.pack(fill="both", expand=True, padx=14, pady=14)

        left = tk.Frame(body, bg=WHITE)
        right = tk.Frame(body, bg=WHITE)
        left.pack(side="left", fill="both", expand=True, padx=(0, 10))
        right.pack(side="left", fill="both", expand=True)

        # LEFT 카드
        left_card = tk.Frame(left, bg=SILVER, highlightbackground=SILVER2, highlightthickness=1)
        left_card.pack(fill="both", expand=True)

        bal_bar = tk.Frame(left_card, bg=SILVER)
        bal_bar.pack(fill="x", padx=12, pady=(12, 8))

        self.var_balance = tk.StringVar(value="잔액 로딩중...")
        tk.Label(bal_bar, textvariable=self.var_balance, bg=SILVER, fg=BLACK,
                 font=("맑은 고딕", 12, "bold")).pack(side="left")

        btn_wrap = tk.Frame(bal_bar, bg=SILVER)
        btn_wrap.pack(side="right")

        tk.Button(btn_wrap, text="원화충전", bg=BLUE, fg=WHITE, bd=0,
                  font=("맑은 고딕", 11, "bold"), command=self.open_krw_usd_charge_popup,
                  width=10).pack(side="left", padx=(0, 6))

        tk.Button(btn_wrap, text="CASH충전", bg=BLACK, fg=WHITE, bd=0,
                  font=("맑은 고딕", 11, "bold"), command=self.open_cash_charge_popup,
                  width=10).pack(side="left", padx=(0, 6))

        tk.Button(btn_wrap, text="유료회원", bg=BLACK, fg=WHITE, bd=0,
                  font=("맑은 고딕", 11, "bold"), command=self.open_membership_popup,
                  width=10).pack(side="left")

        tk.Label(left_card, text="추천 종목", bg=SILVER, fg=BLACK,
                 font=("맑은 고딕", 13, "bold")).pack(anchor="w", padx=12, pady=(0, 6))

        # 추천 탭
        self.nb_reco = ttk.Notebook(left_card)
        self.nb_reco.pack(fill="x", padx=10, pady=(0, 10))

        tab_kr = tk.Frame(self.nb_reco, bg=WHITE)
        tab_us = tk.Frame(self.nb_reco, bg=WHITE)
        self.nb_reco.add(tab_kr, text="국내 TOP10")
        self.nb_reco.add(tab_us, text="미국 TOP10")

        cols = ("종목", "티커", "현재가", "거래대금", "모멘텀(7)", "판단")

        def make_tree(parent, height):
            tree = ttk.Treeview(parent, columns=cols, show="headings", height=height)
            for c in cols:
                tree.heading(c, text=c)
            tree.column("종목", width=180, anchor="w")
            tree.column("티커", width=110, anchor="center")
            tree.column("현재가", width=110, anchor="e")
            tree.column("거래대금", width=140, anchor="e")
            tree.column("모멘텀(7)", width=90, anchor="center")
            tree.column("판단", width=80, anchor="center")
            tree.tag_configure("매수", foreground=GREEN)
            tree.tag_configure("매도", foreground=RED)
            tree.tag_configure("관망", foreground=BLACK)

            vsb = ttk.Scrollbar(parent, orient="vertical", command=tree.yview)
            tree.configure(yscrollcommand=vsb.set)
            tree.pack(side="left", fill="both", expand=True, padx=(8, 0), pady=8)
            vsb.pack(side="right", fill="y", padx=(0, 8), pady=8)
            return tree

        # 높이 반으로
        self.tree_kr = make_tree(tab_kr, height=6)
        self.tree_us = make_tree(tab_us, height=6)

        for name, ticker in RECO_KR_TOP10:
            self.tree_kr.insert("", "end", iid=f"KR:{ticker}",
                                values=(name, ticker, "-", "-", "-", "관망"),
                                tags=("관망",))
        for name, ticker in RECO_US_TOP10:
            self.tree_us.insert("", "end", iid=f"US:{ticker}",
                                values=(name, ticker, "-", "-", "-", "관망"),
                                tags=("관망",))

        self.tree_kr.bind("<Button-1>", self.on_reco_single_click)
        self.tree_us.bind("<Button-1>", self.on_reco_single_click)

        # 구매내역/수익내역
        tk.Label(left_card, text="내역", bg=SILVER, fg=BLACK,
                 font=("맑은 고딕", 13, "bold")).pack(anchor="w", padx=12, pady=(0, 6))

        self.nb_hist = ttk.Notebook(left_card)
        self.nb_hist.pack(fill="both", expand=True, padx=10, pady=(0, 10))

        tab_orders = tk.Frame(self.nb_hist, bg=WHITE)
        tab_pnl = tk.Frame(self.nb_hist, bg=WHITE)
        self.nb_hist.add(tab_orders, text="구매내역")
        self.nb_hist.add(tab_pnl, text="수익내역")

        # 구매내역 테이블
        ocols = ("시간", "매수/매도", "종목", "티커", "수량", "단가", "금액")
        self.tree_orders = ttk.Treeview(tab_orders, columns=ocols, show="headings", height=10)
        for c in ocols:
            self.tree_orders.heading(c, text=c)
        self.tree_orders.column("시간", width=140, anchor="center")
        self.tree_orders.column("매수/매도", width=80, anchor="center")
        self.tree_orders.column("종목", width=160, anchor="w")
        self.tree_orders.column("티커", width=110, anchor="center")
        self.tree_orders.column("수량", width=90, anchor="e")
        self.tree_orders.column("단가", width=90, anchor="e")
        self.tree_orders.column("금액", width=120, anchor="e")
        self.tree_orders.pack(fill="both", expand=True, padx=8, pady=8)

        # 수익내역(보유/평가손익)
        pcols = ("종목", "티커", "보유수량", "평단", "현재가", "평가손익", "평가금액")
        self.tree_pnl = ttk.Treeview(tab_pnl, columns=pcols, show="headings", height=10)
        for c in pcols:
            self.tree_pnl.heading(c, text=c)
        self.tree_pnl.column("종목", width=160, anchor="w")
        self.tree_pnl.column("티커", width=110, anchor="center")
        self.tree_pnl.column("보유수량", width=90, anchor="e")
        self.tree_pnl.column("평단", width=90, anchor="e")
        self.tree_pnl.column("현재가", width=90, anchor="e")
        self.tree_pnl.column("평가손익", width=110, anchor="e")
        self.tree_pnl.column("평가금액", width=120, anchor="e")
        self.tree_pnl.pack(fill="both", expand=True, padx=8, pady=8)

        # RIGHT 카드
        right_card = tk.Frame(right, bg=SILVER, highlightbackground=SILVER2, highlightthickness=1)
        right_card.pack(fill="both", expand=True)

        self.nb_right = ttk.Notebook(right_card)
        self.nb_right.pack(fill="both", expand=True, padx=10, pady=10)

        self.tab_chart = tk.Frame(self.nb_right, bg=WHITE)
        self.tab_board = tk.Frame(self.nb_right, bg=WHITE)
        self.tab_chat  = tk.Frame(self.nb_right, bg=WHITE)

        self.nb_right.add(self.tab_chart, text="차트")
        self.nb_right.add(self.tab_board, text="게시판")
        self.nb_right.add(self.tab_chat,  text="채팅방")
        self.nb_right.bind("<<NotebookTabChanged>>", self.on_right_tab_changed)

        # 차트 탭 자리
        self.var_chart_title = tk.StringVar(value="종목을 원클릭하면 매수/매도 팝업 + 차트가 표시됩니다.")
        tk.Label(self.tab_chart, textvariable=self.var_chart_title,
                 bg=WHITE, fg=BLACK, font=("맑은 고딕", 12, "bold")).pack(anchor="w", padx=10, pady=(10, 0))
        self.chart_box = tk.Frame(self.tab_chart, bg=WHITE)
        self.chart_box.pack(fill="both", expand=True, padx=10, pady=10)

        # 게시판 자리
        self.board_box = tk.Frame(self.tab_board, bg=WHITE)
        self.board_box.pack(fill="both", expand=True)

        # 채팅 자리
        self.chat_box = tk.Frame(self.tab_chat, bg=WHITE)
        self.chat_box.pack(fill="both", expand=True)

        # 하단 안내
        footer = tk.Frame(self, bg=WHITE)
        footer.pack(fill="x", padx=14, pady=(0, 10))
        tk.Label(footer, text=BOTTOM_TEXT, bg=WHITE, fg="#111827",
                 font=("맑은 고딕", 10, "bold"), justify="left").pack(anchor="w")

        self.refresh_top_and_balance()
        self.refresh_reco_async()
        self.refresh_hist_views()

    def refresh_top_and_balance(self):
        if self.session.get("is_guest") and self.session.get("guest_expires"):
            if (self.session["guest_expires"] - datetime.now()).total_seconds() <= 0:
                self._expire_to_login()
                return
            self.var_top.set(self._guest_left_text())
        else:
            uid = self.session.get("user_id", "")
            sub_until = (self.session.get("profile") or {}).get("subscription_until")
            sub_txt = f" | 유료~ {sub_until}" if sub_until else ""

            if self.session.get("auth_pending"):
                attempt = int(self.session.get("auth_retry_count", 0) or 0)
                err = self.session.get("auth_last_error")
                if self.session.get("auth_slow"):
                    if err:
                        self.var_top.set(f"{uid} 로그인 확인 지연(재시도중 {attempt}){sub_txt} | 마지막오류: {err}")
                    else:
                        self.var_top.set(f"{uid} 로그인 확인 지연(재시도중 {attempt}){sub_txt}")
                else:
                    self.var_top.set(f"{uid} 로그인 확인중...{sub_txt}")
            else:
                if self.session.get("auth_verified"):
                    self.var_top.set(f"{uid} 로그인됨{sub_txt}  (로그아웃: Ctrl+Shift+L)")
                else:
                    self.var_top.set(f"{uid} 로그인 상태 미확정{sub_txt}")

        p = self.session.get("profile") or {}
        krw = float(p.get("balance_krw", 0.0) or 0.0)
        usd = float(p.get("balance_usd", 0.0) or 0.0)
        cash = float(p.get("balance_cash", 0.0) or 0.0)
        total = self.calc_total_assets_krw()
        self.var_balance.set(f"총자산 {total:,.0f}원 | KRW {krw:,.0f} | USD {usd:,.2f} | CASH {cash:,.0f}")

    # -------------------------------------------------------------------------
    # charges 로그(공통)
    # -------------------------------------------------------------------------
    def _write_charge_log_tx(self, transaction, db, charge_doc_id: str, payload: dict):
        from firebase_admin import firestore
        cref = db.collection(COL_CHARGES).document(charge_doc_id)
        snap = cref.get(transaction=transaction)
        if snap.exists:
            # 중복 기록 방지
            return
        payload = payload or {}
        if "created_at" not in payload:
            payload["created_at"] = firestore.SERVER_TIMESTAMP
        transaction.set(cref, payload, merge=True)

    # -------------------------------------------------------------------------
    # 원화충전(KRW/USD) + charges 로그
    # -------------------------------------------------------------------------
    def open_krw_usd_charge_popup(self):
        win = tk.Toplevel(self)
        win.title("원화충전")
        win.geometry("520x340")
        win.configure(bg=WHITE)

        tk.Label(win, text="원화충전", bg=WHITE, fg=BLACK,
                 font=("맑은 고딕", 18, "bold")).pack(pady=14)

        frm = tk.Frame(win, bg=WHITE)
        frm.pack(padx=18, pady=10, fill="x")

        var_ccy = tk.StringVar(value="KRW")

        tk.Label(frm, text="통화 선택", bg=WHITE).grid(row=0, column=0, sticky="w", pady=6)
        rb = tk.Frame(frm, bg=WHITE)
        rb.grid(row=0, column=1, sticky="w", pady=6)
        tk.Radiobutton(rb, text="KRW", variable=var_ccy, value="KRW", bg=WHITE).pack(side="left", padx=6)
        tk.Radiobutton(rb, text="USD", variable=var_ccy, value="USD", bg=WHITE).pack(side="left", padx=6)

        tk.Label(frm, text="금액 입력", bg=WHITE).grid(row=1, column=0, sticky="w", pady=6)
        ent = ttk.Entry(frm, width=22)
        ent.grid(row=1, column=1, sticky="w", pady=6)

        def apply_charge():
            try:
                amt = float(ent.get().replace(",", "").strip())
                if amt <= 0:
                    raise ValueError("양의 숫자만 입력")
            except Exception as e:
                messagebox.showwarning("입력 오류", f"금액을 정확히 입력하세요: {e}")
                return

            ccy = var_ccy.get()
            p = self.session["profile"]

            if ccy == "KRW":
                p["balance_krw"] = float(p.get("balance_krw", 0.0) or 0.0) + amt
            else:
                p["balance_usd"] = float(p.get("balance_usd", 0.0) or 0.0) + amt

            self.refresh_top_and_balance()

            # DB 저장(회원만) + charges 기록(트랜잭션)
            if (not self.session.get("is_guest")) and self.session.get("user_id"):
                # 검증 완료 전에는 충전 반영만 하고, DB 저장은 검증 완료 후 허용
                if not self.session.get("auth_verified"):
                    messagebox.showwarning("로그인 확인중", "계정 검증 완료 후 DB 저장됩니다.\n(현재 화면에는 선반영)")
                uid = self.session["user_id"]

                def worker():
                    try:
                        def run_once():
                            db = get_db()
                            from firebase_admin import firestore

                            mem_ref = db.collection(COL_MEMBERS).document(uid)
                            ch_id = "m_" + uuid.uuid4().hex[:12]

                            @firestore.transactional
                            def _tx(transaction):
                                transaction.set(mem_ref, {
                                    "balance_krw": p.get("balance_krw", 0.0),
                                    "balance_usd": p.get("balance_usd", 0.0),
                                }, merge=True)

                                self._write_charge_log_tx(transaction, db, ch_id, {
                                    "id": ch_id,
                                    "user_id": uid,
                                    "method": "MANUAL",
                                    "kind": "KRWUSD_CHARGE",
                                    "ccy": ccy,
                                    "amount": float(amt),
                                    "time": _now_str(),
                                })

                            tx = db.transaction()
                            _tx(tx)

                        retry_op(run_once, tries=5, base_delay=0.6)
                    except Exception as e:
                        self.ui_q.put(("UI_WARN", ("DB 저장 실패", f"충전 저장 오류:\n{e}")))
                        db_log_event_async("ERROR", "krwusd_charge_save_fail", str(e), user_id=uid)

                threading.Thread(target=worker, daemon=True).start()

            messagebox.showinfo("완료", f"{ccy} +{amt:,.2f} 반영")
            win.destroy()

        tk.Button(win, text="충전하기", bg=BLUE, fg=WHITE, bd=0,
                  font=("맑은 고딕", 12, "bold"), command=apply_charge).pack(pady=18)

        tk.Label(win, text="※ 원화충전은 유료결제가 아닙니다. (KRW/USD 잔액 반영 + charges 로그 기록)", bg=WHITE, fg="#374151").pack(pady=(0, 8))

    # -------------------------------------------------------------------------
    # PayPal 결제 공통(자동검증): create->approve->return->capture->반영 + charges 로그
    # -------------------------------------------------------------------------
    def start_paypal_payment_flow(self, purpose: str, amount_krw: float, on_success_apply):
        """
        purpose: 'CASH' or 'SUB'
        amount_krw: 결제 기준 KRW (USD로 환산해 PayPal 결제)
        on_success_apply(order_id, captured_json): 결제완료 후 반영 콜백 -> members patch dict 반환
        """
        if self.session.get("is_guest"):
            messagebox.showwarning("게스트", "게스트는 결제/유료 기능이 제한됩니다.")
            return
        if not self.session.get("user_id"):
            return
        if not self.session.get("auth_verified"):
            messagebox.showwarning("로그인 확인중", "계정 검증이 완료되면 결제를 진행할 수 있습니다.\n(지연 시 자동 재시도 중)")
            return

        usd_value = max(0.5, float(amount_krw) / float(USD_TO_KRW))
        usd_str = f"{usd_value:.2f}"

        self.show_loading("PayPal", "결제 준비중...", 10)

        def worker():
            server = PayPalLocalReturnServer()
            try:
                server.start()
                return_url = f"http://127.0.0.1:{server.port}/return"
                cancel_url = f"http://127.0.0.1:{server.port}/cancel"

                self.ui_q.put(("LOADING", (None, "PayPal", "토큰 발급중...", 25)))
                access = PAYPAL.get_access_token()
                if not access:
                    raise RuntimeError("PayPal access_token 발급 실패")

                self.ui_q.put(("LOADING", (None, "PayPal", "주문 생성중...", 45)))
                order = PAYPAL.create_order(
                    access_token=access,
                    amount_value=usd_str,
                    currency="USD",
                    return_url=return_url,
                    cancel_url=cancel_url
                )

                order_id = order.get("id")
                links = order.get("links", []) or []
                approve_url = None
                for lk in links:
                    if lk.get("rel") == "approve":
                        approve_url = lk.get("href")
                        break
                if not order_id or not approve_url:
                    raise RuntimeError("PayPal 주문 생성 응답이 비정상입니다.")

                self.ui_q.put(("PAYPAL_OPEN", (approve_url, usd_str, purpose)))

                self.ui_q.put(("LOADING", (None, "PayPal", "결제 완료 대기중(브라우저)", 70)))
                token = server.wait_token(timeout=300)

                if token == "__CANCEL__":
                    self.ui_q.put(("PAYPAL_CANCEL", None))
                    return
                if token in ("__TIMEOUT__", "", None):
                    raise RuntimeError("결제 확인 타임아웃(5분). 다시 시도하세요.")

                self.ui_q.put(("LOADING", (None, "PayPal", "결제 캡처/검증중...", 85)))
                captured = PAYPAL.capture_order(access_token=access, order_id=token)
                status = (captured or {}).get("status")
                if status != "COMPLETED":
                    raise RuntimeError(f"결제 상태가 COMPLETED가 아닙니다: {status}")

                uid = self.session["user_id"]
                captured_safe = _compact_json_for_firestore(captured)

                def apply_once():
                    db = get_db()
                    from firebase_admin import firestore

                    pay_ref = db.collection(COL_PAYPAL).document(token)
                    ch_ref = db.collection(COL_CHARGES).document(token)  # ✅ order_id로 charges 중복 방지
                    mem_ref = db.collection(COL_MEMBERS).document(uid)

                    @firestore.transactional
                    def _tx(transaction):
                        snap = pay_ref.get(transaction=transaction)
                        if snap.exists and (snap.to_dict() or {}).get("applied") is True:
                            return

                        transaction.set(pay_ref, {
                            "order_id": token,
                            "user_id": uid,
                            "purpose": purpose,
                            "usd": usd_str,
                            "amount_krw": float(amount_krw),
                            "status": "COMPLETED",
                            "applied": True,
                            "captured": captured_safe,
                            "time": _now_str(),
                            "created_at": firestore.SERVER_TIMESTAMP
                        }, merge=True)

                        # ✅ charges 로그
                        snapc = ch_ref.get(transaction=transaction)
                        if not snapc.exists:
                            transaction.set(ch_ref, {
                                "id": token,
                                "user_id": uid,
                                "method": "PAYPAL",
                                "kind": "CASH_CHARGE" if purpose == "CASH" else "SUBSCRIPTION",
                                "ccy": "KRW",
                                "amount": float(amount_krw),
                                "usd": usd_str,
                                "status": "COMPLETED",
                                "time": _now_str(),
                                "created_at": firestore.SERVER_TIMESTAMP,
                            }, merge=True)

                        patch = on_success_apply(token, captured) or {}
                        if patch:
                            transaction.set(mem_ref, patch, merge=True)

                    tx = db.transaction()
                    _tx(tx)

                retry_op(apply_once, tries=5, base_delay=0.6)
                self.ui_q.put(("PAYPAL_OK", (token, purpose)))

            except Exception as e:
                self.ui_q.put(("PAYPAL_FAIL", str(e)))
                db_log_event_async("ERROR", "paypal_flow_fail", str(e), user_id=self.session.get("user_id",""))
            finally:
                try:
                    server.stop()
                except Exception:
                    pass

        threading.Thread(target=worker, daemon=True).start()

    # -------------------------------------------------------------------------
    # CASH 유료충전(자동검증)
    # -------------------------------------------------------------------------
    def open_cash_charge_popup(self):
        win = tk.Toplevel(self)
        win.title("CASH 유료충전")
        win.geometry("560x420")
        win.configure(bg=WHITE)

        tk.Label(win, text="CASH 유료충전", bg=WHITE, fg=BLACK,
                 font=("맑은 고딕", 18, "bold")).pack(pady=14)

        tk.Label(win, text="CASH는 별풍선/유료동영상 시청 등에 사용하는 유료 포인트입니다.\n"
                           "결제 완료 시 자동으로 CASH가 충전됩니다(자동 검증).",
                 bg=WHITE, fg="#374151", font=("맑은 고딕", 11, "bold")).pack(pady=(0, 10))

        tk.Label(win, text="PayPal 안내(참고): paypal.me/thest8888\n"
                           "※ 자동검증은 PayPal Checkout로 진행됩니다.",
                 bg=WHITE, fg="#111827", font=("맑은 고딕", 10, "bold")).pack(pady=(0, 10))

        frm = tk.Frame(win, bg=WHITE)
        frm.pack(padx=18, pady=10, fill="x")

        tk.Label(frm, text="충전할 CASH(KRW 기준)", bg=WHITE).grid(row=0, column=0, sticky="w", pady=6)
        ent = ttk.Entry(frm, width=22)
        ent.grid(row=0, column=1, sticky="w", pady=6)
        ent.insert(0, "10000")

        def apply_cash():
            try:
                amt_krw = float(ent.get().replace(",", "").strip())
                if amt_krw <= 0:
                    raise ValueError("양의 숫자만 입력")
            except Exception as e:
                messagebox.showwarning("입력 오류", f"금액을 정확히 입력하세요: {e}")
                return

            def on_success_apply(order_id, captured_json):
                p = self.session["profile"]
                p["balance_cash"] = float(p.get("balance_cash", 0.0) or 0.0) + amt_krw
                self.ui_q.put(("REFRESH_BAL", None))
                return {"balance_cash": p.get("balance_cash", 0.0)}

            self.start_paypal_payment_flow("CASH", amt_krw, on_success_apply)
            win.destroy()

        tk.Button(win, text="PayPal로 결제 시작(자동검증)", bg=BLACK, fg=WHITE, bd=0,
                  font=("맑은 고딕", 12, "bold"), command=apply_cash).pack(pady=14)

    # -------------------------------------------------------------------------
    # 유료회원 결제(자동검증) + 날짜 연장
    # -------------------------------------------------------------------------
    def open_membership_popup(self):
        win = tk.Toplevel(self)
        win.title("유료회원 가입/연장")
        win.geometry("560x420")
        win.configure(bg=WHITE)

        tk.Label(win, text="유료회원 가입/연장", bg=WHITE, fg=BLACK,
                 font=("맑은 고딕", 18, "bold")).pack(pady=14)

        tk.Label(win, text="결제 완료되면 자동으로 유효기간이 연장됩니다(자동 검증).",
                 bg=WHITE, fg="#374151", font=("맑은 고딕", 11, "bold")).pack(pady=(0, 10))

        frm = tk.Frame(win, bg=WHITE)
        frm.pack(padx=18, pady=10, fill="x")

        var_plan = tk.StringVar(value="MONTH")
        tk.Radiobutton(frm, text="1달 300,000원", variable=var_plan, value="MONTH", bg=WHITE).pack(anchor="w", pady=6)
        tk.Radiobutton(frm, text="1년 3,000,000원", variable=var_plan, value="YEAR", bg=WHITE).pack(anchor="w", pady=6)

        def start_sub_payment():
            plan = var_plan.get()
            amount_krw = 300000.0 if plan == "MONTH" else 3000000.0
            add_days = 30 if plan == "MONTH" else 365

            def on_success_apply(order_id, captured_json):
                p = self.session["profile"]
                today = datetime.now().date()
                cur = p.get("subscription_until")
                base = today
                try:
                    if cur:
                        cur_d = datetime.strptime(cur, "%Y-%m-%d").date()
                        base = max(today, cur_d)
                except Exception:
                    base = today
                new_d = base + timedelta(days=add_days)
                p["subscription_until"] = new_d.strftime("%Y-%m-%d")
                self.ui_q.put(("REFRESH_BAL", None))
                return {"subscription_until": p["subscription_until"]}

            self.start_paypal_payment_flow("SUB", amount_krw, on_success_apply)
            win.destroy()

        tk.Button(win, text="PayPal 결제 시작(자동검증)", bg=BLUE, fg=WHITE, bd=0,
                  font=("맑은 고딕", 12, "bold"), command=start_sub_payment).pack(pady=14)

        tk.Label(win, text="※ PayPal.me(thest8888)는 참고 링크이며,\n자동검증/자동연장은 Checkout 결제로 처리됩니다.",
                 bg=WHITE, fg="#374151").pack(pady=(8, 0))

    # -------------------------------------------------------------------------
    # 추천 메트릭(비동기)
    # -------------------------------------------------------------------------
    def refresh_reco_async(self):
        def worker():
            self.ui_q.put(("LOADING", (None, "추천데이터", "현재가/거래대금 계산중...", 10)))
            tickers = [t for _, t in RECO_KR_TOP10] + [t for _, t in RECO_US_TOP10]
            total = len(tickers)
            for i, tkc in enumerate(tickers):
                try:
                    d = fetch_metrics_fast(tkc)
                    self.ui_q.put(("METRIC_ONE", (tkc, d)))
                except Exception:
                    pass
                pct = 10 + int((i + 1) / max(1, total) * 85)
                self.ui_q.put(("LOADING", (None, "추천데이터", f"{i+1}/{total} 갱신중...", pct)))
            self.ui_q.put(("LOADING", (None, "추천데이터", "완료", 100)))

            self.ui_q.put(("REFRESH_BAL", None))
            self.ui_q.put(("REFRESH_HIST", None))

        threading.Thread(target=worker, daemon=True).start()

    def _apply_metric_to_tree(self, ticker: str, data: dict):
        price = data.get("price")
        tv = data.get("trade_value")
        up7 = data.get("up7")
        sig = data.get("signal", "관망")

        iid_kr = f"KR:{ticker}"
        iid_us = f"US:{ticker}"

        if self.tree_kr.exists(iid_kr):
            cur = list(self.tree_kr.item(iid_kr, "values"))
            cur[2] = _fmt_price(ticker, price)
            cur[3] = _fmt_trade_value(ticker, tv)
            cur[4] = "-" if up7 is None else str(up7)
            cur[5] = sig
            self.tree_kr.item(iid_kr, values=cur, tags=(sig,))
        elif self.tree_us.exists(iid_us):
            cur = list(self.tree_us.item(iid_us, "values"))
            cur[2] = _fmt_price(ticker, price)
            cur[3] = _fmt_trade_value(ticker, tv)
            cur[4] = "-" if up7 is None else str(up7)
            cur[5] = sig
            self.tree_us.item(iid_us, values=cur, tags=(sig,))

    # -------------------------------------------------------------------------
    # 추천 원클릭 팝업 + 차트 로드
    # -------------------------------------------------------------------------
    def on_reco_single_click(self, e):
        tree = e.widget
        if not isinstance(tree, ttk.Treeview):
            return

        row_id = tree.identify_row(e.y)
        if not row_id:
            return

        region = tree.identify("region", e.x, e.y)
        if region != "cell":
            return

        tree.selection_set(row_id)
        tree.focus(row_id)

        vals = tree.item(row_id, "values")
        if not vals or len(vals) < 2:
            return

        name, ticker = vals[0], vals[1]

        win = self.open_trade_popup(name, ticker)
        if win is None:
            return

        try:
            win.transient(self)
            win.attributes("-topmost", True)
            win.lift()
            win.focus_force()
            win.after(800, lambda: win.attributes("-topmost", False))
        except Exception:
            pass

        self.nb_right.select(self.tab_chart)
        self.load_chart_async(name, ticker)

    # -------------------------------------------------------------------------
    # 매수/매도 팝업 (Firestore 트랜잭션 + retry) + v47 버그 수정
    # -------------------------------------------------------------------------
    def open_trade_popup(self, name: str, ticker: str):
        if (not self.session.get("is_guest")) and (not self.session.get("auth_verified")):
            messagebox.showwarning("로그인 확인중", "계정 검증이 완료되면 거래가 가능합니다.\n(네트워크 지연 시 자동 재시도 중)")
            return None

        popup_id = uuid.uuid4().hex[:10]

        win = tk.Toplevel(self)
        win.title(f"매수/매도 - {name}")
        win.geometry("560x520")
        win.configure(bg=WHITE)

        tk.Label(win, text=f"{name} ({ticker})", bg=WHITE, fg=BLACK,
                 font=("맑은 고딕", 16, "bold")).pack(pady=(14, 4))

        # ✅ v47: 팝업별 var로 분리(여러 팝업 열어도 꼬임 방지)
        trade_price_var = tk.StringVar(value="현재가: 로딩중...")
        tk.Label(win, textvariable=trade_price_var, bg=WHITE, fg="#111827",
                 font=("맑은 고딕", 12, "bold")).pack(pady=(0, 10))

        frm = tk.Frame(win, bg=WHITE)
        frm.pack(padx=18, pady=6, fill="x")

        tk.Label(frm, text="수량", bg=WHITE).grid(row=0, column=0, sticky="w", pady=6)
        ent_qty = ttk.Entry(frm, width=20)
        ent_qty.grid(row=0, column=1, sticky="w", pady=6)

        p = self.session.get("profile") or {}
        is_kr = _is_korean_ticker(ticker)

        state = {"price": None}

        # 캐시 우선
        cached = get_cached_metrics(ticker)
        if cached and cached.get("price") is not None:
            state["price"] = float(cached["price"])
            unit = "KRW" if is_kr else "USD"
            trade_price_var.set(f"현재가: {_fmt_price(ticker, state['price'])} {unit}")
        else:
            def price_worker():
                d = fetch_metrics_fast(ticker)
                self.ui_q.put(("TRADE_PRICE", {"popup_id": popup_id, "ticker": ticker, "price": d.get("price")}))

            threading.Thread(target=price_worker, daemon=True).start()

        # 팝업 레지스트리 등록
        self._trade_popups[popup_id] = {"var": trade_price_var, "ticker": ticker, "win": win, "state": state}

        def _on_close():
            try:
                self._trade_popups.pop(popup_id, None)
            except Exception:
                pass
            try:
                win.destroy()
            except Exception:
                pass

        win.protocol("WM_DELETE_WINDOW", _on_close)

        def current_budget():
            krw = float(p.get("balance_krw", 0.0) or 0.0)
            usd = float(p.get("balance_usd", 0.0) or 0.0)
            cash = float(p.get("balance_cash", 0.0) or 0.0)
            buy_budget = (krw + cash) if is_kr else usd
            ccy = "KRW" if is_kr else "USD"
            return buy_budget, ccy, krw, usd, cash

        def max_qty():
            px = state["price"]
            if px is None or float(px) <= 0:
                return 0.0
            buy_budget, _ccy, _krw, _usd, _cash = current_budget()
            return max(0.0, float(buy_budget) / float(px))

        def set_pct(pct: float):
            q = max_qty() * pct
            ent_qty.delete(0, tk.END)
            if is_kr:
                ent_qty.insert(0, f"{q:.0f}")
            else:
                ent_qty.insert(0, f"{q:.4f}")

        btnbar = tk.Frame(win, bg=WHITE)
        btnbar.pack(pady=(2, 10))

        tk.Button(btnbar, text="5%", bg=SILVER, bd=0, command=lambda: set_pct(0.05)).pack(side="left", padx=4)
        tk.Button(btnbar, text="10%", bg=SILVER, bd=0, command=lambda: set_pct(0.10)).pack(side="left", padx=4)
        tk.Button(btnbar, text="50%", bg=SILVER, bd=0, command=lambda: set_pct(0.50)).pack(side="left", padx=4)
        tk.Button(btnbar, text="전액", bg=SILVER, bd=0, command=lambda: set_pct(1.00)).pack(side="left", padx=4)

        def place_order(side: str):
            try:
                q = float(ent_qty.get().replace(",", "").strip())
                if q <= 0:
                    raise ValueError
            except Exception:
                messagebox.showwarning("입력", "수량을 정확히 입력")
                return

            px = float(state["price"]) if state["price"] is not None else 0.0
            if px <= 0:
                messagebox.showwarning("가격", "현재가를 불러오지 못했습니다. 잠시 후 다시 시도하세요.")
                return

            # ✅ v47: 매도 보유수량 체크
            if side == "SELL":
                hold = float((self.positions.get(ticker, {}) or {}).get("qty", 0.0) or 0.0)
                if hold <= 0:
                    messagebox.showwarning("보유수량", "보유수량이 없어 매도할 수 없습니다.")
                    return
                if q > hold + 1e-12:
                    messagebox.showwarning("보유수량", f"보유수량({hold:g})보다 많이 매도할 수 없습니다.")
                    return

            amt = q * px
            buy_budget, ccy, krw, usd, cash = current_budget()

            order = {
                "id": "o_" + uuid.uuid4().hex[:10],
                "created_at": _now_str(),
                "user_id": self.session.get("user_id", ""),
                "side": side,
                "name": name,
                "ticker": ticker,
                "qty": q,
                "price": px,
                "ccy": ccy,
                "amount": amt
            }

            # 즉시 로컬 반영(빠른 UI)
            if side == "BUY":
                if is_kr:
                    if (krw + cash) < amt:
                        messagebox.showwarning("잔액", "KRW/CASH 잔액 부족")
                        return
                    use_cash = min(cash, amt)
                    remain = amt - use_cash
                    p["balance_cash"] = cash - use_cash
                    p["balance_krw"] = krw - remain
                else:
                    if usd < amt:
                        messagebox.showwarning("잔액", "USD 잔액 부족")
                        return
                    p["balance_usd"] = usd - amt
            else:
                if is_kr:
                    p["balance_krw"] = krw + amt
                else:
                    p["balance_usd"] = usd + amt

            self.orders_cache.insert(0, order)
            self.orders_cache = self.orders_cache[:50]
            self.rebuild_positions_from_orders()
            self.refresh_hist_views()
            self.refresh_top_and_balance()

            # DB 저장(회원만)
            if (not self.session.get("is_guest")) and self.session.get("user_id"):
                uid = self.session["user_id"]

                def worker():
                    try:
                        def run_once():
                            db = get_db()
                            from firebase_admin import firestore

                            order_ref = db.collection(COL_ORDERS).document(order["id"])
                            mem_ref = db.collection(COL_MEMBERS).document(uid)

                            @firestore.transactional
                            def _tx(transaction):
                                transaction.set(order_ref, order, merge=True)
                                transaction.set(mem_ref, {
                                    "balance_krw": p.get("balance_krw", 0.0),
                                    "balance_usd": p.get("balance_usd", 0.0),
                                    "balance_cash": p.get("balance_cash", 0.0)
                                }, merge=True)

                            tx = db.transaction()
                            _tx(tx)

                        retry_op(run_once, tries=5, base_delay=0.6)
                    except Exception as e:
                        self.ui_q.put(("UI_WARN", ("DB 저장 실패", f"주문 저장 오류:\n{e}")))
                        db_log_event_async("ERROR", "order_save_fail", str(e), user_id=uid)

                threading.Thread(target=worker, daemon=True).start()

            messagebox.showinfo("주문", f"{'매수' if side=='BUY' else '매도'} 주문 처리\n{ticker} {q} @ {px}")
            _on_close()

        act = tk.Frame(win, bg=WHITE)
        act.pack(pady=10)

        tk.Button(act, text="매수", bg=GREEN, fg=WHITE, bd=0,
                  font=("맑은 고딕", 12, "bold"), width=10,
                  command=lambda: place_order("BUY")).pack(side="left", padx=8)

        tk.Button(act, text="매도", bg=RED, fg=WHITE, bd=0,
                  font=("맑은 고딕", 12, "bold"), width=10,
                  command=lambda: place_order("SELL")).pack(side="left", padx=8)

        tk.Label(win, text="※ 총자산은 보유종목 평가금액까지 포함합니다.\n"
                           "※ 보유수량/평단은 주문 기반으로 계산됩니다(간단 평균법).",
                 bg=WHITE, fg="#374151").pack(pady=(8, 0))

        return win

    # -------------------------------------------------------------------------
    # 내역 UI 갱신
    # -------------------------------------------------------------------------
    def refresh_hist_views(self):
        try:
            if hasattr(self, "tree_orders"):
                for i in self.tree_orders.get_children():
                    self.tree_orders.delete(i)
                for o in (self.orders_cache or [])[:30]:
                    side = o.get("side")
                    self.tree_orders.insert("", "end", values=(
                        o.get("created_at",""),
                        "매수" if side=="BUY" else "매도",
                        o.get("name",""),
                        o.get("ticker",""),
                        f"{float(o.get('qty',0.0) or 0.0):,.4f}".rstrip("0").rstrip("."),
                        f"{float(o.get('price',0.0) or 0.0):,.2f}",
                        f"{float(o.get('amount',0.0) or 0.0):,.2f}",
                    ))

            if hasattr(self, "tree_pnl"):
                for i in self.tree_pnl.get_children():
                    self.tree_pnl.delete(i)

                for ticker, info in (self.positions or {}).items():
                    qty = float(info.get("qty",0.0) or 0.0)
                    avg = float(info.get("avg_cost",0.0) or 0.0)
                    m = get_cached_metrics(ticker) or {}
                    px = m.get("price")
                    if px is None:
                        cur = "-"
                        pnl = "-"
                        val = "-"
                    else:
                        px = float(px)
                        cur = _fmt_price(ticker, px)
                        pnl_v = (px - avg) * qty
                        val_v = px * qty
                        if info.get("ccy") == "USD":
                            pnl = f"{pnl_v:,.2f} USD"
                            val = f"{val_v:,.2f} USD"
                        else:
                            pnl = f"{pnl_v:,.0f} KRW"
                            val = f"{val_v:,.0f} KRW"

                    self.tree_pnl.insert("", "end", values=(
                        info.get("name",""),
                        ticker,
                        f"{qty:,.4f}".rstrip("0").rstrip("."),
                        f"{avg:,.2f}",
                        cur,
                        pnl,
                        val
                    ))
        except Exception:
            pass

    # -------------------------------------------------------------------------
    # 오른쪽 탭 변경: Lazy Load
    # -------------------------------------------------------------------------
    def on_right_tab_changed(self, _e):
        cur = self.nb_right.index(self.nb_right.select())
        title = self.nb_right.tab(cur, "text")
        if title == "차트" and not self._chart_ready:
            self.init_chart_tab()
        elif title == "게시판" and not self._board_ready:
            self.init_board_tab()
        elif title == "채팅방" and not self._chat_ready:
            self.init_chat_tab()

    # -------------------------------------------------------------------------
    # 차트 탭 초기화/로드(타임아웃으로 멈춤 방지)
    # -------------------------------------------------------------------------
    def init_chart_tab(self):
        try:
            setup_matplotlib_korean_font()
        except Exception:
            pass

        try:
            from matplotlib.figure import Figure
            from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

            self._mpl_fig = Figure(figsize=(8.2, 5.0), dpi=100)
            self._mpl_ax = self._mpl_fig.add_subplot(111)
            self._mpl_ax.set_title("")

            self._mpl_canvas = FigureCanvasTkAgg(self._mpl_fig, master=self.chart_box)
            self._mpl_canvas.get_tk_widget().pack(fill="both", expand=True)
            self._chart_ready = True

        except Exception as e:
            self._chart_ready = True
            messagebox.showwarning("차트", f"matplotlib 초기화 실패: {e}")

    def load_chart_async(self, name: str, ticker: str):
        if not self._chart_ready:
            self.init_chart_tab()

        self.var_chart_title.set(f"{name} ({ticker}) 차트 로딩중...")
        self.show_loading("차트", "데이터 다운로드...", 15)

        def worker():
            try:
                self.ui_q.put(("LOADING", (None, "차트", "데이터 다운로드...", 35)))
                df = fetch_chart_data(ticker)
                self.ui_q.put(("CHART_OK", {"name": name, "ticker": ticker, "df": df}))
                self.ui_q.put(("LOADING", (None, "차트", "렌더링...", 90)))
                self.ui_q.put(("LOADING", (None, "차트", "완료", 100)))
            except Exception as e:
                self.ui_q.put(("CHART_FAIL", str(e)))

        threading.Thread(target=worker, daemon=True).start()

    def render_chart(self, name: str, ticker: str, df):
        if not self._mpl_ax or not self._mpl_canvas:
            return

        if df is None or df.empty:
            self.var_chart_title.set(f"{name} ({ticker}) 차트 없음(데이터 지연)")
            return

        self._mpl_ax.clear()
        self._mpl_ax.plot(df.index, df["Close"])
        self._mpl_ax.set_title(f"{name} ({ticker})")

        try:
            self._mpl_fig.autofmt_xdate()
        except Exception:
            pass

        self._mpl_canvas.draw()
        self.var_chart_title.set(f"{name} ({ticker}) 차트")

    # -------------------------------------------------------------------------
    # 게시판 (방송 실시간 알림 포함)
    # -------------------------------------------------------------------------
    def ensure_board_collection(self):
        if self.session.get("is_guest"):
            return
        try:
            db = get_db()
            from firebase_admin import firestore
            docref = db.collection(COL_BOARD).document("__schema__board_posts")
            docref.set({
                "title": "__schema__",
                "body": "board_posts 컬렉션 초기화/스키마 문서",
                "user_id": "SYSTEM",
                "time": "2000-01-01 00:00:00",
                "created_at": firestore.SERVER_TIMESTAMP
            }, merge=True)
        except Exception:
            pass

    def init_board_tab(self):
        # 방송(실시간 알림) 영역
        bc_top = tk.Frame(self.board_box, bg=WHITE)
        bc_top.pack(fill="x", padx=10, pady=(10, 0))

        tk.Label(bc_top, text="방송(실시간 알림)", bg=WHITE, fg=BLACK,
                 font=("맑은 고딕", 11, "bold")).pack(side="left")

        self.broadcast_list = tk.Listbox(self.board_box, height=4, font=("맑은 고딕", 10))
        self.broadcast_list.pack(fill="x", padx=10, pady=(6, 10))
        self.broadcast_list.insert(tk.END, "방송 대기중...")

        def on_broadcast(payload):
            self.ui_q.put(("BOARD_BROADCAST", payload))

        # ✅ 중복 구독 방지
        if "방송" not in self._subscribed_rooms:
            FASTBUS.subscribe("방송", on_broadcast)
            self._subscribed_rooms.add("방송")

        top = tk.Frame(self.board_box, bg=WHITE)
        top.pack(fill="x", padx=10, pady=10)

        tk.Label(top, text="게시판", bg=WHITE, fg=BLACK,
                 font=("맑은 고딕", 13, "bold")).pack(side="left")

        tk.Button(top, text="더보기", bg=BLACK, fg=WHITE, bd=0,
                  font=("맑은 고딕", 10, "bold"),
                  command=self.board_load_more, width=8).pack(side="right", padx=(6, 0))

        tk.Button(top, text="새로고침", bg=BLUE, fg=WHITE, bd=0,
                  font=("맑은 고딕", 10, "bold"),
                  command=self.board_refresh, width=8).pack(side="right", padx=(0, 6))

        self.board_list = tk.Listbox(self.board_box, height=14, font=("맑은 고딕", 11))
        self.board_list.pack(fill="both", expand=True, padx=10)
        self.board_list.bind("<Double-Button-1>", self.board_open_selected)

        frm = tk.Frame(self.board_box, bg=WHITE)
        frm.pack(fill="x", padx=10, pady=10)

        self.ent_board_title = ttk.Entry(frm)
        self.ent_board_title.pack(side="left", fill="x", expand=True, padx=(0, 8))
        self.ent_board_title.insert(0, "제목")

        def _title_focus_in(_e):
            if self.ent_board_title.get().strip() == "제목":
                self.ent_board_title.delete(0, tk.END)

        def _title_focus_out(_e):
            if not self.ent_board_title.get().strip():
                self.ent_board_title.insert(0, "제목")

        self.ent_board_title.bind("<FocusIn>", _title_focus_in)
        self.ent_board_title.bind("<FocusOut>", _title_focus_out)

        tk.Button(frm, text="글쓰기", bg=BLACK, fg=WHITE, bd=0,
                  font=("맑은 고딕", 10, "bold"),
                  command=self.board_write).pack(side="right")

        self.txt_board_body = ScrolledText(self.board_box, height=6, font=("맑은 고딕", 11))
        self.txt_board_body.pack(fill="x", padx=10, pady=(0, 10))
        self.txt_board_body.insert("end", "내용...")

        def _body_focus_in(_e):
            if self.txt_board_body.get("1.0", tk.END).strip() == "내용...":
                self.txt_board_body.delete("1.0", tk.END)

        def _body_focus_out(_e):
            if not self.txt_board_body.get("1.0", tk.END).strip():
                self.txt_board_body.insert("end", "내용...")

        self.txt_board_body.bind("<FocusIn>", _body_focus_in)
        self.txt_board_body.bind("<FocusOut>", _body_focus_out)

        self._board_ready = True
        self.ensure_board_collection()
        self.board_refresh()

    def board_refresh(self):
        self.board_page = 0
        self.board_last_doc = None
        if hasattr(self, "board_list"):
            self.board_list.delete(0, tk.END)
        self.board_load_page(append=False)

    def board_load_more(self):
        self.board_page += 1
        self.board_load_page(append=True)

    def board_load_page(self, append=False):
        if self.session.get("is_guest"):
            if hasattr(self, "board_list") and (not append):
                self.board_list.delete(0, tk.END)
            if hasattr(self, "board_list"):
                self.board_list.insert(tk.END, "[게스트] 게시판은 읽기만 제공(다음 버전에 확장)")
            return

        def worker():
            try:
                self.ui_q.put(("LOADING", (None, "게시판", "DB에서 글 목록 불러오는중...", 20)))
                db = get_db()
                from firebase_admin import firestore

                def _query_created_at():
                    q = (db.collection(COL_BOARD)
                            .order_by("created_at", direction=firestore.Query.DESCENDING)
                            .limit(self.board_page_size))
                    if append and self.board_last_doc is not None:
                        q = q.start_after(self.board_last_doc)
                    return list(q.stream())

                def _query_time_fallback():
                    # created_at 인덱스/필드 문제 시 fallback
                    q = (db.collection(COL_BOARD)
                            .order_by("time", direction=firestore.Query.DESCENDING)
                            .limit(self.board_page_size))
                    if append and self.board_last_doc is not None:
                        q = q.start_after(self.board_last_doc)
                    return list(q.stream())

                try:
                    docs = _query_created_at()
                except Exception:
                    docs = _query_time_fallback()

                items = []
                last_doc = None

                for doc in docs:
                    if doc.id.startswith("__schema__"):
                        continue
                    d = doc.to_dict() or {}
                    items.append({
                        "id": doc.id,
                        "title": d.get("title", "(제목없음)"),
                        "user": d.get("user_id", ""),
                        "time": d.get("time", "")
                    })
                    last_doc = doc

                self.ui_q.put(("BOARD_CURSOR", last_doc))
                self.ui_q.put(("BOARD_LIST_APPEND" if append else "BOARD_LIST", items))
                self.ui_q.put(("LOADING", (None, "게시판", "완료", 100)))
            except Exception as e:
                self.ui_q.put(("BOARD_ERR", str(e)))
                db_log_event_async("ERROR", "board_load_fail", str(e), user_id=self.session.get("user_id",""))

        threading.Thread(target=worker, daemon=True).start()

    def board_write(self):
        if self.session.get("is_guest"):
            messagebox.showwarning("게스트", "게스트는 글쓰기 제한(다음 버전에서 해제 가능)")
            return

        uid = self.session.get("user_id", "")
        if not uid:
            return

        title = self.ent_board_title.get().strip()
        body = self.txt_board_body.get("1.0", tk.END).strip()

        # ✅ v47: placeholder 저장 방지
        if (not title) or title == "제목":
            messagebox.showwarning("입력", "제목을 입력하세요.")
            return
        if (not body) or body == "내용...":
            messagebox.showwarning("입력", "내용을 입력하세요.")
            return

        def worker():
            try:
                self.ui_q.put(("LOADING", (None, "게시판", "글 저장중...", 30)))
                db = get_db()
                from firebase_admin import firestore

                pid = "p_" + uuid.uuid4().hex[:10]
                now_s = _now_str()

                def run_once():
                    db.collection(COL_BOARD).document(pid).set({
                        "title": title,
                        "body": body,
                        "user_id": uid,
                        "time": now_s,
                        "created_at": firestore.SERVER_TIMESTAMP
                    }, merge=True)

                retry_op(run_once, tries=5, base_delay=0.6)

                # ✅ v47: 방송 송출(FastBus)
                FASTBUS.publish("방송", {
                    "type": "board",
                    "room": "방송",
                    "time": datetime.now().strftime("%H:%M:%S"),
                    "user": uid,
                    "title": title,
                    "post_id": pid
                })

                self.ui_q.put(("LOADING", (None, "게시판", "완료", 100)))
                self.ui_q.put(("BOARD_REFRESH", None))
            except Exception as e:
                self.ui_q.put(("BOARD_ERR", str(e)))
                db_log_event_async("ERROR", "board_write_fail", str(e), user_id=uid)

        threading.Thread(target=worker, daemon=True).start()

    def board_open_selected(self, _e=None):
        sel = self.board_list.curselection()
        if not sel:
            return

        line = self.board_list.get(sel[0])
        if line.startswith("[") or line.strip() in ("(글이 없습니다)", "(더 불러올 글이 없습니다)"):
            return
        if " | id:" not in line:
            return

        pid = line.split(" | id:", 1)[1].strip()

        def worker():
            try:
                self.ui_q.put(("LOADING", (None, "게시판", "글 불러오는중...", 30)))
                db = get_db()
                doc = db.collection(COL_BOARD).document(pid).get()
                if not doc.exists:
                    self.ui_q.put(("BOARD_ERR", "글이 없습니다"))
                    return
                d = doc.to_dict() or {}
                self.ui_q.put(("BOARD_OPEN", {
                    "title": d.get("title", ""),
                    "body": d.get("body", ""),
                    "user": d.get("user_id", ""),
                    "time": d.get("time", "")
                }))
                self.ui_q.put(("LOADING", (None, "게시판", "완료", 100)))
            except Exception as e:
                self.ui_q.put(("BOARD_ERR", str(e)))
                db_log_event_async("ERROR", "board_open_fail", str(e), user_id=self.session.get("user_id",""))

        threading.Thread(target=worker, daemon=True).start()

    # -------------------------------------------------------------------------
    # 채팅방
    # -------------------------------------------------------------------------
    def init_chat_tab(self):
        top = tk.Frame(self.chat_box, bg=WHITE)
        top.pack(fill="x", padx=10, pady=10)

        tk.Label(top, text="채팅방(초고속)", bg=WHITE, fg=BLACK,
                 font=("맑은 고딕", 13, "bold")).pack(side="left")

        tk.Button(top, text="방 만들기", bg=BLUE, fg=WHITE, bd=0,
                  font=("맑은 고딕", 10, "bold"), command=self.chat_create_room).pack(side="right", padx=6)
        tk.Button(top, text="방 입장", bg=BLACK, fg=WHITE, bd=0,
                  font=("맑은 고딕", 10, "bold"), command=self.chat_join_room).pack(side="right")

        self.room_list = tk.Listbox(self.chat_box, height=7, font=("맑은 고딕", 11))
        self.room_list.pack(fill="x", padx=10)

        self._rooms = ["전체방", "인기방1", "인기방2", "방송"]
        self._render_rooms()

        self.chat_log = ScrolledText(self.chat_box, height=12, font=("맑은 고딕", 11), wrap="word")
        self.chat_log.pack(fill="both", expand=True, padx=10, pady=10)
        self.chat_log.insert("end", "방에 입장하면 대화가 시작됩니다.\n")

        emo = tk.Frame(self.chat_box, bg=WHITE)
        emo.pack(fill="x", padx=10, pady=(0, 8))
        for em in EMOTES:
            tk.Button(emo, text=em, bg=SILVER, bd=0, font=("맑은 고딕", 14),
                      command=lambda x=em: self.chat_entry.insert(tk.END, x)).pack(side="left", padx=2)

        frm = tk.Frame(self.chat_box, bg=WHITE)
        frm.pack(fill="x", padx=10, pady=(0, 12))

        self.chat_entry = ttk.Entry(frm, font=("맑은 고딕", 12))
        self.chat_entry.pack(side="left", fill="x", expand=True, padx=(0, 8))

        tk.Button(frm, text="별풍선", bg="#FF6B6B", fg=WHITE, bd=0,
                  font=("맑은 고딕", 10, "bold"),
                  command=self.chat_send_balloon, width=8).pack(side="right", padx=4)
        tk.Button(frm, text="전송", bg=BLUE, fg=WHITE, bd=0,
                  font=("맑은 고딕", 10, "bold"),
                  command=self.chat_send_message, width=8).pack(side="right")

        self._chat_ready = True
        self._subscribe_room("전체방")

    def _render_rooms(self):
        self.room_list.delete(0, tk.END)
        for r in self._rooms:
            self.room_list.insert(tk.END, r)

    def chat_create_room(self):
        name = simpledialog.askstring("방 만들기", "방 이름을 입력하세요")
        if not name:
            return
        name = name.strip()
        if not name:
            return
        if name in self._rooms:
            messagebox.showwarning("중복", "이미 존재하는 방입니다")
            return
        self._rooms.append(name)
        self._render_rooms()
        messagebox.showinfo("완료", f"방 생성: {name}")

    def chat_join_room(self):
        sel = self.room_list.curselection()
        if not sel:
            return
        room = self.room_list.get(sel[0]).strip()
        if not room:
            return
        self._subscribe_room(room)
        self.chat_log.insert("end", f"\n[입장] {room}\n")
        self.chat_log.see(tk.END)

    def _subscribe_room(self, room: str):
        self.current_room = room

        if room in self._subscribed_rooms:
            return

        def on_msg(payload):
            self.ui_q.put(("CHAT_MSG", payload))
        FASTBUS.subscribe(room, on_msg)
        self._subscribed_rooms.add(room)

    def chat_send_message(self):
        msg = self.chat_entry.get().strip()
        if not msg:
            return
        user = self.session.get("user_id") or ("GUEST" if self.session.get("is_guest") else "익명")
        payload = {
            "type": "chat",
            "room": self.current_room,
            "user": user,
            "time": datetime.now().strftime("%H:%M:%S"),
            "message": msg
        }
        FASTBUS.publish(self.current_room, payload)
        self.chat_entry.delete(0, tk.END)

    def chat_send_balloon(self):
        user = self.session.get("user_id") or ("GUEST" if self.session.get("is_guest") else "익명")
        payload = {
            "type": "gift",
            "room": self.current_room,
            "user": user,
            "time": datetime.now().strftime("%H:%M:%S"),
            "message": BALLOON_GIFT
        }
        FASTBUS.publish(self.current_room, payload)

    # -------------------------------------------------------------------------
    # Firestore에서 최근 주문내역 로드
    # -------------------------------------------------------------------------
    def load_orders_async(self, token, uid: str):
        def worker():
            try:
                if token is not getattr(self, "_login_token", None):
                    return
                db = get_db()
                from firebase_admin import firestore
                q = (db.collection(COL_ORDERS)
                        .where("user_id", "==", uid)
                        .order_by("created_at", direction=firestore.Query.DESCENDING)
                        .limit(50))
                docs = list(q.stream())
                items = []
                for d in docs:
                    items.append(d.to_dict() or {})
                self.ui_q.put(("ORDERS_LOADED", {"token": token, "items": items}))
            except Exception as e:
                self.ui_q.put(("UI_WARN", ("주문내역 로드 실패", str(e))))
                db_log_event_async("ERROR", "orders_load_fail", str(e), user_id=uid)

        threading.Thread(target=worker, daemon=True).start()

    # -------------------------------------------------------------------------
    # Tick
    # -------------------------------------------------------------------------
    def _tick(self):
        if self.session.get("is_guest") and self.session.get("guest_expires"):
            if (self.session["guest_expires"] - datetime.now()).total_seconds() <= 0:
                self._expire_to_login()
                return
            if hasattr(self, "var_top"):
                self.refresh_top_and_balance()

        while not self.ui_q.empty():
            typ, payload = self.ui_q.get()

            if typ == "LOADING":
                token, title, text, pct = payload
                # 로그인 로딩은 token 체크(로그아웃/재로그인 후 잔여 로딩 무시)
                if token is not None and token is not getattr(self, "_login_token", None):
                    continue
                self.show_loading(title, text, pct)
                if int(pct) >= 100:
                    self.after(150, self.hide_loading)

            elif typ == "UI_INFO":
                title, msg = payload
                messagebox.showinfo(title, msg)

            elif typ == "SIGNUP_DONE":
                try:
                    if hasattr(self, "_signup_win") and self._signup_win and self._signup_win.winfo_exists():
                        self._signup_win.destroy()
                except Exception:
                    pass

            elif typ == "AUTH_SLOW":
                token = payload.get("token")
                if token is not getattr(self, "_login_token", None):
                    continue
                self.refresh_top_and_balance()

            elif typ == "AUTH_RETRYING":
                # ✅ v47: token 불일치/이미 성공(auth_verified True)이면 무시
                token = payload.get("token")
                if token is not getattr(self, "_login_token", None):
                    continue
                if self.session.get("auth_verified"):
                    continue
                self.hide_loading()
                self.session["auth_pending"] = True
                self.session["auth_slow"] = True
                self.refresh_top_and_balance()

            elif typ == "LOGIN_OK":
                token = payload.get("token")
                if token is not getattr(self, "_login_token", None):
                    continue

                self.hide_loading()

                self.session["logged_in"] = True
                self.session["is_guest"] = False
                self.session["user_id"] = payload["uid"]
                self.session["profile"] = payload["profile"] or {}

                self.session["auth_pending"] = False
                self.session["auth_start_ts"] = None
                self.session["auth_verified"] = True
                self.session["auth_retry_count"] = 0
                self.session["auth_last_error"] = None
                self.session["auth_slow"] = False

                self.refresh_top_and_balance()

            elif typ == "LOGIN_FAIL_FATAL":
                token = payload.get("token")
                if token is not getattr(self, "_login_token", None):
                    continue
                self.hide_loading()
                messagebox.showwarning("로그인 실패", payload.get("err", "로그인 실패"))

                try:
                    self.save_auto_login("", "", enabled=False)
                except Exception:
                    pass

                self._expire_to_login()

            elif typ == "LOAD_ORDERS":
                token = payload.get("token")
                uid = payload.get("uid")
                if token is not getattr(self, "_login_token", None):
                    continue
                self.load_orders_async(token, uid)

            elif typ == "ORDERS_LOADED":
                token = payload.get("token")
                if token is not getattr(self, "_login_token", None):
                    continue
                self.orders_cache = payload.get("items") or []
                self.rebuild_positions_from_orders()
                self.refresh_hist_views()
                self.refresh_top_and_balance()

            elif typ == "REFRESH_BAL":
                self.refresh_top_and_balance()

            elif typ == "REFRESH_HIST":
                self.refresh_hist_views()

            elif typ == "UI_WARN":
                title, msg = payload
                messagebox.showwarning(title, msg)

            elif typ == "METRIC_ONE":
                ticker, data = payload
                try:
                    self._apply_metric_to_tree(ticker, data)
                except Exception:
                    pass

            elif typ == "PAYPAL_OPEN":
                approve_url, usd_str, purpose = payload
                self.hide_loading()
                webbrowser.open(approve_url)
                messagebox.showinfo("PayPal 결제", f"브라우저에서 결제를 완료하세요.\n(목적: {purpose}, USD: {usd_str})")

            elif typ == "PAYPAL_OK":
                order_id, purpose = payload
                self.hide_loading()
                self.refresh_top_and_balance()
                messagebox.showinfo("결제 완료", f"자동 검증 완료!\norder_id: {order_id}\n목적: {purpose}")

            elif typ == "PAYPAL_CANCEL":
                self.hide_loading()
                messagebox.showwarning("결제 취소", "결제가 취소되었습니다.")

            elif typ == "PAYPAL_FAIL":
                self.hide_loading()
                messagebox.showwarning("결제 실패", f"{payload}\n\n환경변수(PAYPAL_CLIENT_ID/SECRET) 확인하세요.")

            elif typ == "CHART_OK":
                try:
                    self.hide_loading()
                    name = payload["name"]
                    ticker = payload["ticker"]
                    df = payload["df"]
                    self.render_chart(name, ticker, df)
                except Exception:
                    pass

            elif typ == "CHART_FAIL":
                self.hide_loading()
                messagebox.showwarning("차트", f"차트 로딩 실패: {payload}")

            elif typ == "BOARD_CURSOR":
                self.board_last_doc = payload

            elif typ == "BOARD_LIST":
                self.hide_loading()
                items = payload or []
                if not hasattr(self, "board_list"):
                    continue
                self.board_list.delete(0, tk.END)
                if not items:
                    self.board_list.insert(tk.END, "(글이 없습니다)")
                else:
                    for it in items:
                        line = f"{it['title']} | {it.get('user','')} | {it.get('time','')} | id:{it['id']}"
                        self.board_list.insert(tk.END, line)

            elif typ == "BOARD_LIST_APPEND":
                self.hide_loading()
                items = payload or []
                if not hasattr(self, "board_list"):
                    continue
                if not items:
                    self.board_list.insert(tk.END, "(더 불러올 글이 없습니다)")
                else:
                    for it in items:
                        line = f"{it['title']} | {it.get('user','')} | {it.get('time','')} | id:{it['id']}"
                        self.board_list.insert(tk.END, line)

            elif typ == "BOARD_REFRESH":
                self.hide_loading()
                self.board_refresh()

            elif typ == "BOARD_OPEN":
                self.hide_loading()
                d = payload
                win = tk.Toplevel(self)
                win.title("게시글 보기")
                win.geometry("640x520")
                win.configure(bg=WHITE)

                tk.Label(win, text=d.get("title",""), bg=WHITE, fg=BLACK,
                         font=("맑은 고딕", 14, "bold")).pack(pady=10)
                tk.Label(win, text=f"{d.get('user','')} | {d.get('time','')}",
                         bg=WHITE, fg="#374151").pack(pady=(0, 10))

                txt = ScrolledText(win, font=("맑은 고딕", 11), wrap="word")
                txt.pack(fill="both", expand=True, padx=10, pady=10)
                txt.insert("end", d.get("body",""))
                txt.configure(state="disabled")

            elif typ == "BOARD_ERR":
                self.hide_loading()
                messagebox.showwarning("게시판", payload)

            elif typ == "BOARD_BROADCAST":
                # 방송 리스트 업데이트
                if not hasattr(self, "broadcast_list"):
                    continue
                p = payload or {}
                if p.get("type") == "board":
                    t = p.get("time","")
                    u = p.get("user","")
                    title = p.get("title","")
                    s = f"[{t}] {u}: {title}"
                    self._broadcast_items.append(s)
                    self._broadcast_items = self._broadcast_items[-30:]
                    self.broadcast_list.delete(0, tk.END)
                    for x in self._broadcast_items[-20:]:
                        self.broadcast_list.insert(tk.END, x)

            elif typ == "CHAT_MSG":
                if not hasattr(self, "chat_log"):
                    continue
                p = payload
                t = p.get("time","")
                u = p.get("user","")
                m = p.get("message","")
                self.chat_log.insert("end", f"[{t}] {u}: {m}\n")
                self.chat_log.see(tk.END)

            elif typ == "TRADE_PRICE":
                # ✅ v47: popup_id로 해당 팝업만 업데이트
                popup_id = payload.get("popup_id")
                info = self._trade_popups.get(popup_id)
                if not info:
                    continue
                price = payload.get("price")
                ticker = payload.get("ticker","")
                try:
                    if price is None:
                        info["var"].set("현재가: 로딩 실패")
                    else:
                        px = float(price)
                        info["state"]["price"] = px
                        unit = "KRW" if _is_korean_ticker(ticker) else "USD"
                        info["var"].set(f"현재가: {_fmt_price(ticker, px)} {unit}")
                except Exception:
                    pass

        self.after(160, self._tick)


# =============================================================================
# 실행
# =============================================================================
def main():
    app = STAIv47()
    app.mainloop()


if __name__ == "__main__":
    main()
